# Google

In [114]:
import os, requests
import pandas as pd
from dotenv import load_dotenv


file_types = {
    "문서 파일": {
        "PDF": "pdf",
        "Word (DOC)": "doc",
        "Word (DOCX)": "docx",
        "엑셀 (XLS)": "xls",
        "엑셀 (XLSX)": "xlsx",
        "PowerPoint (PPT)": "ppt",
        "PowerPoint (PPTX)": "pptx",
        "텍스트 (TXT)": "txt",
        "OpenDocument 텍스트 (ODT)": "odt",
        "RTF (서식 있는 텍스트)": "rtf"
    },
    "이미지 파일": {
        "JPEG": "jpg",
        "JPEG (또는 JPG)": "jpeg",
        "PNG": "png",
        "GIF": "gif",
        "BMP": "bmp",
        "TIFF": "tiff"
    },
    "오디오/비디오 파일": {
        "MP3 (오디오)": "mp3",
        "WAV (오디오)": "wav",
        "MP4 (비디오)": "mp4",
        "AVI (비디오)": "avi",
        "MOV (비디오)": "mov"
    },
    "압축 파일": {
        "ZIP": "zip",
        "RAR": "rar",
        "TAR": "tar",
        "7z": "7z"
    },
    "코드/스크립트 파일": {
        "Python": "py",
        "HTML": "html",
        "HTML (HTM)": "htm",
        "JavaScript": "js",
        "CSS": "css",
        "XML": "xml",
        "JSON": "json",
        "CSV": "csv"
    },
    "기타 파일": {
        "Adobe Illustrator": "ai",
        "Photoshop": "psd",
        "3D 모델 (OBJ)": "obj"
    }
}


load_dotenv()
Google_API_KEY = os.getenv("Google_API_KEY") # 키 생성/가져오기: https://developers.google.com/custom-search/v1/introduction?hl=ko
Google_SEARCH_ENGINE_ID = os.getenv("Google_SEARCH_ENGINE_ID") # 검생 엔진 생성: https://programmablesearchengine.google.com/controlpanel/all

excluded_links = ["tistory", "kin", "youtube", "blog", "book", "news", "dcinside", "fmkorea", "ruliweb", "theqoo", "clien", "mlbpark", "instiz", "todayhumor"]


def google_api(query:str, k:int=100, types:list[str|None]=[]) -> pd.DataFrame:
    """
    Parameters:
        query : 검색하고 싶은 검색어 
        k : 원하는 검색 결과 수 (1 ~ 300)
        types: 원하는 검색 파일 유형 목록

    Returns:
        df_google : 검색 결과

        n | Title | Link | Description
        ---|---|---|---
        0 | 제목 | www.*.com | 설명
        . | ... | ... | ...
        k-1 | 제목 | www.*.com | 설명
    """

    results = []
    url = "https://www.googleapis.com/customsearch/v1"
    query = query.replace("|", "OR")
    if types:
        file_types_li = sum([[*value.values()] for value in file_types.values()], [])
        query += " OR ".join([f"filetype:{type_}" for type_ in types if type_ in file_types_li]) # types에 있는 파일 형식만 검색
    params = {
        "key": Google_API_KEY,
        "cx": Google_SEARCH_ENGINE_ID,
        "q": query,
        "start": 0
    }
    k, count, flag = min(300, max(int(k), 1)), 0, False

    for page in range(1, k+1000, 10): # 페이지는 1부터 시작해서 각각 10개의 결과물을 보여줌.
        params["start"] = page
        response = requests.get(url, params=params)

        if(status_code:= response.status_code) != 200:
            print(f"Error code: {status_code}")
            continue
        items = response.json().get("items", [])

        for item in items:
            link = item.get("link", "")
            if not link or any(ex in link for ex in excluded_links): continue
            title = item.get("title", "no title")
            description = item.get("snippet", "no description")
            results.append([title, link, description] )
            count += 1
            if count >= k: flag = True; break
        if flag: break
            
    df_google = pd.DataFrame(results, columns=['Title', 'Link', 'Description'])
    return df_google


In [11]:
{'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - k-pop', 'totalResults': '2960000000', 'searchTerms': 'k-pop', 'count': 10, 'startIndex': 1, 'inputEncoding': 'utf8', 'outputEncoding': 'utf8', 'safe': 'off', 'cx': '944d504795edf4eed'}], 'nextPage': [{'title': 'Google Custom Search - k-pop', 'totalResults': '2960000000', 'searchTerms': 'k-pop', 'count': 10, 'startIndex': 11, 'inputEncoding': 'utf8', 'outputEncoding': 'utf8', 'safe': 'off', 'cx': '944d504795edf4eed'}]}, 'context': {'title': 'scrape'}, 'searchInformation': {'searchTime': 0.157891, 'formattedSearchTime': '0.16', 'totalResults': '2960000000', 'formattedTotalResults': '2,960,000,000'}, 'items': [{'kind': 'customsearch#result', 'title': 'K-pop - Wikipedia', 'htmlTitle': '<b>K</b>-<b>pop</b> - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/K-pop', 'displayLink': 'en.wikipedia.org', 'snippet': 'It includes styles and genres from around the world, such as pop, hip hop, R&B, rock, jazz, gospel, reggae, electronic dance, folk, country, disco, and\xa0...', 'htmlSnippet': 'It includes styles and genres from around the world, such as <b>pop</b>, hip hop, R&amp;B, rock, jazz, gospel, reggae, electronic dance, folk, country, disco, and&nbsp;...', 'formattedUrl': 'https://en.wikipedia.org/wiki/K-pop', 'htmlFormattedUrl': 'https://en.wikipedia.org/wiki/<b>K-pop</b>', 'pagemap': {'metatags': [{'referrer': 'origin', 'og:image': 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/e1/K-pop_boybands.jpg/1200px-K-pop_boybands.jpg', 'theme-color': '#eaecf0', 'og:image:width': '1200', 'og:type': 'website', 'viewport': 'width=device-width, initial-scale=1.0, user-scalable=yes, minimum-scale=0.25, maximum-scale=5.0', 'og:title': 'K-pop - Wikipedia', 'og:image:height': '741', 'format-detection': 'telephone=no'}]}}, {'kind': 'customsearch#result', 'title': 'Meet 1VERSE, first K-pop boy band to feature North Korean ...', 'htmlTitle': 'Meet 1VERSE, first <b>K</b>-<b>pop</b> boy band to feature North Korean ...', 'link': 'https://m.koreaherald.com/view.php?ud=20241127050145', 'displayLink': 'm.koreaherald.com', 'snippet': 'Nov 28, 2024 ... From North Korea, US, Japan, four members seek tell their own stories through music The four members of 1VERSE – Hyuk, Seok, Aito and Kenny\xa0...', 'htmlSnippet': 'Nov 28, 2024 <b>...</b> From North Korea, US, Japan, four members seek tell their own stories through music The four members of 1VERSE – Hyuk, Seok, Aito and Kenny&nbsp;...', 'formattedUrl': 'https://m.koreaherald.com/view.php?ud=20241127050145', 'htmlFormattedUrl': 'https://m.koreaherald.com/view.php?ud=20241127050145', 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQE8D7CGSpymmayYKu0XRS2J2WZGOSOCdcm7-g2BQtLQIOtAzBOSzQKUxB5&s', 'width': '275', 'height': '183'}], 'metatags': [{'ssc:cmsid': '20241127050145', 'og:image': 'https://wimg.heraldcorp.com/content/default/2024/11/27/20241127050102_0.jpg', 'og:type': 'article', 'twitter:card': 'summary_large_image', 'twitter:title': 'Meet 1VERSE, first K-pop boy band to feature North Korean defectors', 'article:published_time': '2024-11-28T15:29:08+09:00', 'article:sec_id': '007001', 'ssc:paper_date': '2024-11-29', 'og:title': 'Meet 1VERSE, first K-pop boy band to feature North Korean defectors', 'ssc:create_time': '2024-11-28 15:27:06', 'og:description': 'From North Korea, US, Japan, four members seek tell their own stories through music The four members of 1VERSE – Hyuk, Seok, Aito and Kenny – are about to achieve the first milestone that countless K-pop trainees dream of: an official debut with an album and showcases. This group, tirelessly honing', 'twitter:creator': '@Herald', 'twitter:image': 'https://wimg.heraldcorp.com/content/default/2024/11/27/20241127050102_0.jpg', 'twitter:site': '@Herald', 'viewport': 'width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, target-densitydpi=medium-dpi', 'twitter:description': 'From North Korea, US, Japan, four members seek tell their own stories through music The four members of 1VERSE – Hyuk, Seok, Aito and Kenny – are about to achieve the first milestone that countless K-pop trainees dream of: an official debut with an album and showcases. This group, tirelessly honing', 'og:url': 'https://m.koreaherald.com/article/10011859', 'format-detection': 'telephone=no, address=no, email=no'}], 'cse_image': [{'src': 'https://wimg.heraldcorp.com/content/default/2024/11/27/20241127050102_0.jpg'}]}}, {'kind': 'customsearch#result', 'title': 'K-Pop ON! (온) - 50 Songs | Spotify Playlist', 'htmlTitle': '<b>K</b>-<b>Pop</b> ON! (온) - 50 Songs | Spotify Playlist', 'link': 'https://open.spotify.com/playlist/37i9dQZF1DX9tPFwDMOaN1', 'displayLink': 'open.spotify.com', 'snippet': 'Playlist · K-Pop ON! (온) · 50 items · 5.5M saves.', 'htmlSnippet': 'Playlist · <b>K</b>-<b>Pop</b> ON! (온) · 50 items · 5.5M saves.', 'formattedUrl': 'https://open.spotify.com/playlist/37i9dQZF1DX9tPFwDMOaN1', 'htmlFormattedUrl': 'https://open.spotify.com/playlist/37i9dQZF1DX9tPFwDMOaN1', 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR83kMF2rH2aP1syYsyqPyrD8eb51gHEP3h-p1Hd9Bt11s-CkrqCpRloyM&s', 'width': '225', 'height': '225'}], 'metatags': [{'fb:app_id': '174829003346', 'og:type': 'music.playlist', 'og:site_name': 'Spotify', 'viewport': 'width=device-width, initial-scale=1', 'og:title': 'K-Pop ON! (온)', 'og:url': 'https://open.spotify.com/playlist/37i9dQZF1DX9tPFwDMOaN1', 'og:description': 'Playlist · Spotify · 50 items · 5.5M saves', 'og:restrictions:country:allowed': 'JP'}], 'cse_image': [{'src': 'https://i.scdn.co/image/ab67706f00000002a134c8932ab2b1130f15dfec'}]}}, {'kind': 'customsearch#result', 'title': 'The Top Three K-Pop Songs of November 2024 | The Bias List // K ...', 'htmlTitle': 'The Top Three <b>K</b>-<b>Pop</b> Songs of November 2024 | The Bias List // K ...', 'link': 'https://thebiaslist.com/2024/11/30/the-top-three-k-pop-songs-of-november-2024/', 'displayLink': 'thebiaslist.com', 'snippet': "Nov 30, 2024 ... I look back at my three favorite title tracks by K-pop artists. I take my own ratings into account, but there's a bit of wiggle room as certain songs tend to\xa0...", 'htmlSnippet': 'Nov 30, 2024 <b>...</b> I look back at my three favorite title tracks by <b>K</b>-<b>pop</b> artists. I take my own ratings into account, but there&#39;s a bit of wiggle room as certain songs tend to&nbsp;...', 'formattedUrl': 'https://thebiaslist.com/2024/.../the-top-three-k-pop-songs-of-november-202...', 'htmlFormattedUrl': 'https://thebiaslist.com/2024/.../the-top-three-<b>k-pop</b>-songs-of-november-202...', 'pagemap': {'hcard': [{'fn': 'NishiAnuar', 'nickname': 'NishiAnuar'}, {'fn': 'Annietime21'}, {'fn': 'Annietime21'}, {'url_text': '사이기', 'fn': '사이기', 'nickname': '사이기', 'url': 'http://saighkaillelgmailcom.wordpress.com/'}, {'fn': 'Lalaland', 'nickname': 'Lalaland'}, {'fn': 'Bobson Nogilny Dugnutt, Jr.'}, {'fn': 'Lizen', 'nickname': 'Lizen'}, {'fn': 'Bobson Nogilny Dugnutt, Jr.'}, {'fn': 'Guntherstewart', 'nickname': 'Guntherstewart'}, {'fn': 'matchaman', 'nickname': 'matchaman'}], 'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQNYLwssh04Yzi2jq9Twfioelv1GGVPfdI9VXJ1DsD3Rno26UIWlPgWP7U&s', 'width': '300', 'height': '168'}], 'metatags': [{'application-name': 'The Bias List // K-Pop Reviews & Discussion', 'og:image': 'https://thebiaslist.com/wp-content/uploads/2016/01/monthly-round-up.jpg', 'og:type': 'article', 'article:published_time': '2024-11-30T15:29:04+00:00', 'og:image:width': '450', 'twitter:card': 'summary_large_image', 'og:site_name': 'The Bias List // K-Pop Reviews & Discussion', 'msapplication-window': 'width=device-width;height=device-height', 'msvalidate.01': '397F6191EC9FEF835EFF5DB807A1AF3E', 'msapplication-tooltip': 'K-Pop reviews and discussion with just a hint of bias...', 'og:title': 'The Top Three K-Pop Songs of November 2024', 'og:image:height': '253', 'msapplication-tileimage': 'https://thebiaslist.com/wp-content/uploads/2018/08/cropped-biaslistavatar.jpg?w=162', 'og:description': 'At the end of each\xa0month, I look back at my three favorite title tracks by K-pop artists. I take my own ratings into account, but there’s a bit of wiggle room as certain songs tend to grow or…', 'article:publisher': 'https://www.facebook.com/WordPresscom', 'twitter:image': 'https://thebiaslist.com/wp-content/uploads/2016/01/monthly-round-up.jpg?w=640', 'twitter:text:title': 'The Top Three K-Pop Songs of November 2024', 'article:modified_time': '2024-11-30T15:29:04+00:00', 'viewport': 'width=device-width', 'og:locale': 'en_US', 'og:url': 'https://thebiaslist.com/2024/11/30/the-top-three-k-pop-songs-of-november-2024/'}], 'cse_image': [{'src': 'https://thebiaslist.com/wp-content/uploads/2016/01/monthly-round-up.jpg'}]}}, {'kind': 'customsearch#result', 'title': 'A Brief History of K-pop – The Los Angeles Film School', 'htmlTitle': 'A Brief History of <b>K</b>-<b>pop</b> – The Los Angeles Film School', 'link': 'https://www.lafilm.edu/blog/a-brief-history-of-kpop/', 'displayLink': 'www.lafilm.edu', 'snippet': 'Apr 6, 2021 ... K-pop or Korean Pop is the internationally popular, aesthetic-driven, style-bending, trendsetting, music genre of the 21st century.', 'htmlSnippet': 'Apr 6, 2021 <b>...</b> <b>K</b>-<b>pop</b> or <b>Korean Pop</b> is the internationally popular, aesthetic-driven, style-bending, trendsetting, music genre of the 21st century.', 'formattedUrl': 'https://www.lafilm.edu/blog/a-brief-history-of-kpop/', 'htmlFormattedUrl': 'https://www.lafilm.edu/blog/a-brief-history-of-<b>kpop</b>/', 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ4P5K8eh-KbWDnlSswV6G1ExO0dT0p77oqhWeuxpTRcYmI6ZeL4sPWx4o&s', 'width': '300', 'height': '168'}], 'imageobject': [{'width': '1000', 'url': 'https://www.lafilm.edu/wp-content/uploads/2021/04/K-pop.jpeg', 'height': '562'}, {'url': 'https://www.lafilm.edu/wp-content/uploads/2020/03/lafilm-logo-2.png'}], 'person': [{'name': 'Jourdan Arnaud'}], 'organization': [{'name': 'Jourdan Arnaud'}], 'metatags': [{'og:image': 'https://www.lafilm.edu/wp-content/uploads/2021/04/K-pop.jpeg', 'og:type': 'article', 'article:published_time': '2021-04-07T00:57:19+00:00', 'og:image:width': '1000', 'og:site_name': 'The Los Angeles Film School', 'tec-api-origin': 'https://www.lafilm.edu', 'author': 'Jourdan Arnaud', 'og:title': 'A Brief History of K-pop', 'og:image:height': '562', 'twitter:label1': 'Written by', 'twitter:label2': 'Est. reading time', 'og:image:type': 'image/jpeg', 'og:description': 'Find out when K-pop started gaining popularity from its roots in South Korea. We take you through the past and present of K-pop. Read more.', 'article:publisher': 'https://www.facebook.com/losangelesfilmschool/', 'twitter:data1': 'Jourdan Arnaud', 'twitter:data2': '6 minutes', 'article:modified_time': '2023-02-20T18:22:04+00:00', 'viewport': 'width=device-width, initial-scale=1', 'og:locale': 'en_US', 'og:url': 'https://www.lafilm.edu/blog/a-brief-history-of-kpop/', 'tec-api-version': 'v1'}], 'cse_image': [{'src': 'https://www.lafilm.edu/wp-content/uploads/2021/04/K-pop.jpeg'}], 'blogposting': [{'datemodified': '2023-02-20 10:22:04', 'name': 'A Brief History of K-pop', 'text': 'Image via NickiSwift.com. Photo by Rick Fury/Getty ImagesWhen did K-pop become popular?Musical acts made up of attractive and talented young people have been a reliable recipe for success for...', 'headline': 'A Brief History of K-pop', 'datepublished': '2021-04-06 17:57:19', 'mainentityofpage': 'A Brief History of K-pop'}], 'sitenavigationelement': [{'url': 'Degree Programs'}, {'url': 'Degree Programs'}], 'listitem': [{'name': 'Home', 'position': '1', 'url': 'Home'}, {'name': 'Articles', 'position': '2', 'url': 'Articles'}]}}, {'kind': 'customsearch#result', 'title': 'Like Dionysus. BTS, Classics in K-Pop, and the… | by Yung In Chae ...', 'htmlTitle': 'Like Dionysus. BTS, Classics in <b>K</b>-<b>Pop</b>, and the… | by Yung In Chae ...', 'link': 'https://eidolon.pub/like-dionysus-1d1b8fb428e1', 'displayLink': 'eidolon.pub', 'snippet': 'Oct 29, 2020 ... Just get drunk, like Dionysus. In one hand, a drink, in the other, a thyrsus. Art sloshing in a clear crystal glass. Art is alcohol too, if you drink it you\xa0...', 'htmlSnippet': 'Oct 29, 2020 <b>...</b> Just get drunk, like Dionysus. In one hand, a drink, in the other, a thyrsus. Art sloshing in a clear crystal glass. Art is alcohol too, if you drink it you&nbsp;...', 'formattedUrl': 'https://eidolon.pub/like-dionysus-1d1b8fb428e1', 'htmlFormattedUrl': 'https://eidolon.pub/like-dionysus-1d1b8fb428e1', 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ1_wsqlWqr-dB9TP5eac3HC4hUxCSrJlpgTBUJR_T6lUMWcOAB3zFMXivO&s', 'width': '361', 'height': '139'}], 'metatags': [{'og:image': 'https://miro.medium.com/v2/resize:fit:1200/1*_h4WnpKYXWoXT4BsSu1J1Q.jpeg', 'twitter:app:url:iphone': 'medium://p/1d1b8fb428e1', 'theme-color': '#000000', 'article:published_time': '2020-11-19T04:19:23.289Z', 'twitter:card': 'summary_large_image', 'og:site_name': 'Medium', 'al:android:package': 'com.medium.reader', 'twitter:label1': 'Reading time', 'twitter:app:id:iphone': '828256236', 'title': 'Like Dionysus. BTS, Classics in K-Pop, and the… | by Yung In Chae | EIDOLON', 'al:ios:url': 'medium://p/1d1b8fb428e1', 'og:description': 'BTS, Classics in K-Pop, and the Narcissism of the West', 'twitter:creator': '@yunginchae', 'al:ios:app_store_id': '828256236', 'twitter:data1': '16 min read', 'twitter:site': '@eidolon_journal', 'og:type': 'article', 'twitter:title': 'Like Dionysus', 'al:ios:app_name': 'Medium', 'author': 'Yung In Chae', 'og:title': 'Like Dionysus', 'al:web:url': 'https://eidolon.pub/like-dionysus-1d1b8fb428e1', 'article:author': 'https://medium.com/@ychae', 'twitter:image:src': 'https://miro.medium.com/v2/resize:fit:1200/1*_h4WnpKYXWoXT4BsSu1J1Q.jpeg', 'al:android:url': 'medium://p/1d1b8fb428e1', 'referrer': 'unsafe-url', 'fb:app_id': '542599432471018', 'viewport': 'width=device-width,minimum-scale=1,initial-scale=1,maximum-scale=1', 'twitter:description': 'BTS, Classics in K-Pop, and the Narcissism of the West', 'og:url': 'https://eidolon.pub/like-dionysus-1d1b8fb428e1', 'twitter:app:name:iphone': 'Medium', 'al:android:app_name': 'Medium'}], 'cse_image': [{'src': 'https://miro.medium.com/v2/resize:fit:1200/1*_h4WnpKYXWoXT4BsSu1J1Q.jpeg'}]}}, {'kind': 'customsearch#result', 'title': 'K-Pop has been overusing English in recent years. : r ...', 'htmlTitle': '<b>K</b>-<b>Pop</b> has been overusing English in recent years. : r ...', 'link': 'https://www.reddit.com/r/unpopularkpopopinions/comments/12hqtfd/kpop_has_been_overusing_english_in_recent_years/', 'displayLink': 'www.reddit.com', 'snippet': 'Apr 10, 2023 ... Well, they have to. As Kpop now is a universal phenomenon, adding more and more english, makes it easier for non-korean speaking fans to follow\xa0...', 'htmlSnippet': 'Apr 10, 2023 <b>...</b> Well, they have to. As <b>Kpop</b> now is a universal phenomenon, adding more and more english, makes it easier for non-korean speaking fans to follow&nbsp;...', 'formattedUrl': 'https://www.reddit.com/.../unpopularkpopopinions/.../kpop_has_been_over...', 'htmlFormattedUrl': 'https://www.reddit.com/.../unpopular<b>kpop</b>opinions/.../<b>kpop</b>_has_been_over...', 'pagemap': {'metatags': [{'og:image': 'https://share.redd.it/preview/post/12hqtfd', 'theme-color': '#000000', 'og:image:width': '1200', 'og:type': 'website', 'og:image:alt': 'An image containing a preview of the post', 'twitter:card': 'summary_large_image', 'twitter:title': 'r/unpopularkpopopinions on Reddit: K-Pop has been overusing English in recent years.', 'og:site_name': 'Reddit', 'og:title': 'r/unpopularkpopopinions on Reddit: K-Pop has been overusing English in recent years.', 'og:image:height': '630', 'msapplication-navbutton-color': '#000000', 'og:description': 'Posted by u/Alarod - 118 votes and 152 comments', 'twitter:image': 'https://share.redd.it/preview/post/12hqtfd', 'apple-mobile-web-app-status-bar-style': 'black', 'twitter:site': '@reddit', 'viewport': 'width=device-width, initial-scale=1, viewport-fit=cover', 'apple-mobile-web-app-capable': 'yes', 'og:ttl': '600', 'og:url': 'https://www.reddit.com/r/unpopularkpopopinions/comments/12hqtfd/kpop_has_been_overusing_english_in_recent_years/'}], 'cse_image': [{'src': 'https://share.redd.it/preview/post/12hqtfd'}]}}, {'kind': 'customsearch#result', 'title': 'Cash Monet K-Pop Drag Queen (@cashmonetdrag) • Instagram ...', 'htmlTitle': 'Cash Monet <b>K</b>-<b>Pop</b> Drag Queen (@cashmonetdrag) • Instagram ...', 'link': 'https://www.instagram.com/cashmonetdrag/', 'displayLink': 'www.instagram.com', 'snippet': '10K Followers, 2696 Following, 673 Posts - Cash Monet K-Pop Drag Queen (@cashmonetdrag) on Instagram: "K-Pop Filipino Drag Queen LA Drag Photographer\xa0...', 'htmlSnippet': '10K Followers, 2696 Following, 673 Posts - Cash Monet <b>K</b>-<b>Pop</b> Drag Queen (@cashmonetdrag) on Instagram: &quot;<b>K</b>-<b>Pop</b> Filipino Drag Queen LA Drag Photographer&nbsp;...', 'formattedUrl': 'https://www.instagram.com/cashmonetdrag/', 'htmlFormattedUrl': 'https://www.instagram.com/cashmonetdrag/', 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRXkGh4g8bwsaO-O13rLGEHDMgbJNss4kH6Sr6CarUQdLqWojsnjM_pxIkC&s', 'width': '168', 'height': '300'}], 'metatags': [{'og:image': 'https://scontent-dfw5-2.cdninstagram.com/v/t51.2885-19/436405458_411740781672418_650375357047289007_n.jpg?stp=dst-jpg_s100x100_tt6&_nc_cat=106&ccb=1-7&_nc_sid=bf7eb4&_nc_ohc=OFHC1FZ_j68Q7kNvgHb4D3O&_nc_zt=24&_nc_ht=scontent-dfw5-2.cdninstagram.com&oh=00_AYCA1RoSq_Z83kuURcAt6Kb8J6h4nttoYnOawq1Su89xIQ&oe=678E453E', 'theme-color': '#FFFFFF', 'og:type': 'profile', 'al:ios:app_name': 'Instagram', 'og:title': 'Cash Monet 👠 K-Pop Drag Queen (@cashmonetdrag) • Instagram photos and videos', 'al:android:package': 'com.instagram.android', 'bingbot': 'noarchive', 'al:ios:url': 'instagram://user?username=cashmonetdrag', 'color-scheme': 'light', 'og:description': '10K Followers, 2,696 Following, 673 Posts - See Instagram photos and videos from Cash Monet 👠 K-Pop Drag Queen (@cashmonetdrag)', 'al:ios:app_store_id': '389801252', 'al:android:url': 'https://instagram.com/_u/cashmonetdrag/', 'apple-mobile-web-app-status-bar-style': 'default', 'viewport': 'width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1, viewport-fit=cover', 'mobile-web-app-capable': 'yes', 'og:url': 'https://www.instagram.com/cashmonetdrag/', 'al:android:app_name': 'Instagram'}], 'cse_image': [{'src': 'https://lookaside.instagram.com/seo/google_widget/crawler/?media_id=2882434161709544296'}]}}, {'kind': 'customsearch#result', 'title': 'What are the differences between K-Pop and J-Pop? : r/kpophelp', 'htmlTitle': 'What are the differences between <b>K</b>-<b>Pop</b> and J-Pop? : r/kpophelp', 'link': 'https://www.reddit.com/r/kpophelp/comments/oq7ev4/what_are_the_differences_between_kpop_and_jpop/', 'displayLink': 'www.reddit.com', 'snippet': 'Jul 23, 2021 ... jpop groups at the moment are largely defined by a focus on the individual members and the music, rather than a focus on performance, visuals,\xa0...', 'htmlSnippet': 'Jul 23, 2021 <b>...</b> jpop groups at the moment are largely defined by a focus on the individual members and the music, rather than a focus on performance, visuals,&nbsp;...', 'formattedUrl': 'https://www.reddit.com/.../kpophelp/.../what_are_the_differences_between_...', 'htmlFormattedUrl': 'https://www.reddit.com/.../<b>kpop</b>help/.../what_are_the_differences_between_...', 'pagemap': {'metatags': [{'og:image': 'https://share.redd.it/preview/post/oq7ev4', 'theme-color': '#000000', 'og:image:width': '1200', 'og:type': 'website', 'og:image:alt': 'An image containing a preview of the post', 'twitter:card': 'summary_large_image', 'twitter:title': 'r/kpophelp on Reddit: What are the differences between K-Pop and J-Pop?', 'og:site_name': 'Reddit', 'og:title': 'r/kpophelp on Reddit: What are the differences between K-Pop and J-Pop?', 'og:image:height': '630', 'msapplication-navbutton-color': '#000000', 'og:description': 'Posted by u/Sherllian - 20 votes and 13 comments', 'twitter:image': 'https://share.redd.it/preview/post/oq7ev4', 'apple-mobile-web-app-status-bar-style': 'black', 'twitter:site': '@reddit', 'viewport': 'width=device-width, initial-scale=1, viewport-fit=cover', 'apple-mobile-web-app-capable': 'yes', 'og:ttl': '600', 'og:url': 'https://www.reddit.com/r/kpophelp/comments/oq7ev4/what_are_the_differences_between_kpop_and_jpop/'}], 'cse_image': [{'src': 'https://share.redd.it/preview/post/oq7ev4'}]}}, {'kind': 'customsearch#result', 'title': 'A love letter to K-pop in Atlanta - Atlanta Magazine', 'htmlTitle': 'A love letter to <b>K</b>-<b>pop</b> in Atlanta - Atlanta Magazine', 'link': 'https://www.atlantamagazine.com/news-culture-articles/a-love-letter-to-k-pop-in-atlanta/', 'displayLink': 'www.atlantamagazine.com', 'snippet': 'Dec 7, 2023 ... A love letter to K-pop in Atlanta. With the epicenter in Duluth, metro Atlanta is the K-pop capital of the South.', 'htmlSnippet': 'Dec 7, 2023 <b>...</b> A love letter to <b>K</b>-<b>pop</b> in Atlanta. With the epicenter in Duluth, metro Atlanta is the <b>K</b>-<b>pop</b> capital of the South.', 'formattedUrl': 'https://www.atlantamagazine.com/news.../a-love-letter-to-k-pop-in-atlanta/', 'htmlFormattedUrl': 'https://www.atlantamagazine.com/news.../a-love-letter-to-<b>k-pop</b>-in-atlanta/', 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQbI-hcyYgj-w8fR6YCHQCbrrDYKHphLkM1k9K9_xp11qNpl4_TOOaD79H5&s', 'width': '262', 'height': '192'}], 'imageobject': [{'url': '//cdn2.atlantamagazine.com/wp-content/uploads/sites/4/2018/06/atlanta-logo-black.png'}, {'width': '1200', 'url': 'https://cdn2.atlantamagazine.com/wp-content/uploads/sites/4/2023/12/1123_straykids_oneuseonly.jpg', 'height': '882'}], 'person': [{'name': 'Sara Murphy'}], 'organization': [{'name': 'Atlanta Magazine'}], 'metatags': [{'og:image': 'https://cdn2.atlantamagazine.com/wp-content/uploads/sites/4/2023/12/1123_straykids_oneuseonly.jpg', 'og:type': 'article', 'article:published_time': '2023-12-07T22:28:42+00:00', 'og:image:width': '1200', 'twitter:card': 'summary_large_image', 'twitter:title': 'A love letter to K-pop in Atlanta', 'og:site_name': 'Atlanta Magazine', 'tec-api-origin': 'https://www.atlantamagazine.com', 'og:title': 'A love letter to K-pop in Atlanta', 'og:image:height': '882', 'twitter:label1': 'Written by', 'twitter:label2': 'Est. reading time', 'og:description': 'I’m strolling through Centennial Olympic Park on a warm spring day, rocking a crop top for the first time—at age 42. It’s black with the word MANIAC scrawled in thin, neon pink font. As I soak up the sun, I pass many others in the same shirt. The K-pop group Stray Kids is in town for their “Maniac” world tour, and Atlanta is overrun by STAYs—the official name for Stray Kids fans like me.', 'fb:pages': '41107761812', 'twitter:data1': 'Sara Murphy', 'twitter:data2': '3 minutes', 'fb:app_id': '2287687018133441', 'viewport': 'width=device-width, initial-scale=1.0', 'og:locale': 'en_US', 'fb:admins': '1666521220', 'og:url': 'https://www.atlantamagazine.com/news-culture-articles/a-love-letter-to-k-pop-in-atlanta/', 'tec-api-version': 'v1'}], 'cse_image': [{'src': 'https://cdn2.atlantamagazine.com/wp-content/uploads/sites/4/2023/12/1123_straykids_oneuseonly.jpg'}], 'hatomfeed': [{}], 'article': [{'datemodified': '2023-12-07T17:28:42-05:00', 'headline': 'A love letter to K-pop in Atlanta', 'datepublished': '2023-12-07T17:28:42-05:00'}]}}]}


,Title,Link,Description
0,K-pop - Wikipedia,https://en.wikipedia.org/wiki/K-pop,It includes styles and genres from around the ...
1,"Meet 1VERSE, first K-pop boy band to feature N...",https://m.koreaherald.com/view.php?ud=20241127...,"Nov 28, 2024 ... From North Korea, US, Japan, ..."
2,K-Pop ON! (온) - 50 Songs | Spotify Playlist,https://open.spotify.com/playlist/37i9dQZF1DX9...,Playlist · K-Pop ON! (온) · 50 items · 5.5M saves.
3,The Top Three K-Pop Songs of November 2024 | T...,https://thebiaslist.com/2024/11/30/the-top-thr...,"Nov 30, 2024 ... I look back at my three favor..."
4,"Like Dionysus. BTS, Classics in K-Pop, and the...",https://eidolon.pub/like-dionysus-1d1b8fb428e1,"Oct 29, 2020 ... Just get drunk, like Dionysus..."
5,K-Pop has been overusing English in recent yea...,https://www.reddit.com/r/unpopularkpopopinions...,"Apr 10, 2023 ... Well, they have to. As Kpop n..."
6,Cash Monet K-Pop Drag Queen (@cashmonetdrag) •...,https://www.instagram.com/cashmonetdrag/,"10K Followers, 2696 Following, 673 Posts - Cas..."
7,What are the differences between K-Pop and J-P...,https://www.reddit.com/r/kpophelp/comments/oq7...,"Jul 23, 2021 ... jpop groups at the moment are..."
8,What does it mean to have a bias in k-pop? : r...,https://www.reddit.com/r/NewJeans/comments/122...,"Mar 25, 2023 ... A bias is basically just your..."
9,Why So Many Former Emos Are Now K-Pop Fans,https://www.vice.com/en/article/k-pop-emo-fand...,"Jul 29, 2019 ... Former emos who'd built frien..."


In [113]:
df_google = google_api("kpop", 20)
df_google


headers {'Content-Type': 'application/json; charset=UTF-8', 'Vary': 'Origin, X-Origin, Referer', 'Content-Encoding': 'gzip', 'Date': 'Thu, 16 Jan 2025 10:20:30 GMT', 'Server': 'ESF', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'Transfer-Encoding': 'chunked'}


,Title,Link,Description
0,K-pop - Wikipedia,https://en.wikipedia.org/wiki/K-pop,It includes styles and genres from around the ...


# Naver

In [101]:
import os, re, html, requests
import pandas as pd
from dotenv import load_dotenv


naver_search_api_endpoints  = {
    "news": "뉴스",
    "encyc": "백과사전",
    "blog": "블로그",
    "shop": "쇼핑",
    "webkr": "웹 문서",
    "image": "이미지",
    "doc": "전문정보",
    "kin": "지식iN",
    "book": "책",
    "cafearticle": "카페글",
    "adult": "성인 검색어 판별 결과 반환",
    "errata": "한영 오류 변환 결과 반환",
    "local": "지역별 업체 및 상호"
} # https://developers.naver.com/docs/serviceapi/search/web/web.md#%EC%9B%B9%EB%AC%B8%EC%84%9C

load_dotenv()
Naver_client_id = os.getenv("Naver_client_id")
Naver_client_secret = os.getenv("Naver_client_secret")

excluded_links = ["tistory", "kin", "youtube", "blog", "book", "news", "dcinside", "fmkorea", "ruliweb", "theqoo", "clien", "mlbpark", "instiz", "todayhumor"]


def remove_tags(text:str) -> str:
    tag_pattern = r'<(\/?[\w\s="\':^>]*)>'
    text = re.sub(tag_pattern, '', text).strip()
    text = html.unescape(text)
    return text


def naver_api(query:str, k:int=100, api:str="webkr", display:int=100, sort:str="sim") -> pd.DataFrame:
    """
    Parameters:
        query : 검색하고 싶은 검색어 
        k : 원하는 검색 결과 수 (1 ~ 300)
        api: 검색 api 종류 ('news', 'encyc', 'blog', 'shop', 'webkr', 'image', 'doc', 'kin', 'book', 'cafearticle', 'adult', 'errata', 'local')
        display: 한 번에 표시할 검색 결과 개수
        sort: 검색 결과 정렬 방법 ("sim": 정확도순, "date": 날짜순)

    Returns:
        df_naver : 검색 결과

        n | Title | Link | Description
        ---|---|---|---
        0 | 제목 | www.*.com | 설명
        . | ... | ... | ...
        k-1 | 제목 | www.*.com | 설명
    """

    results = []
    api = api if api in naver_search_api_endpoints else "webkr"
    url = f"https://openapi.naver.com/v1/search/{api}"
    headers = {
        "X-Naver-Client-Id": Naver_client_id,
        "X-Naver-Client-Secret": Naver_client_secret
    }
    # query = urllib.parse.quote(query)
    display = min(100, max(int(display), 10))
    sort = sort if sort in ["sim", "date"] else "sim"
    params = {
        "query": query,
        "display": display,
        "start": 0,
        "sort": sort
    }
    k, count, flag = min(300, max(int(k), 1)), 0, False

    for page in range(1, k+1000, display):
        params["start"] = page
        response = requests.get(url, headers=headers, params=params)
        
        if(status_code:= response.status_code) != 200:
            print(f"Error code: {status_code}")
            continue
        items = response.json().get("items", [])

        for item in items:
            link = item.get('link', '')
            if not link or any(ex in link for ex in excluded_links): continue
            title = item.get('title', 'no title')
            description = item.get('description', 'no description')
            title, description = map(remove_tags, (title, description))
            results.append([title, link, description])
            count += 1
            if count >= k: flag = True; break
        if flag: break

    df_naver = pd.DataFrame(results, columns=['Title', 'Link', 'Description'])
    return df_naver


In [ ]:
{'lastBuildDate': 'Thu, 16 Jan 2025 18:02:05 +0900', 'total': 52194863, 'start': 1, 'display': 10, 'items': [{'title': '<b>육아</b> - 나무위키', 'link': 'https://namu.wiki/w/%EC%9C%A1%EC%95%84', 'description': '育兒 아이를 기르는 것, 보통 미성년자인 자식을 키우는 것을 의미하며 좁게는 초등학생까지의 어린아이를 기르는 것부터 넓게는 성인이 된 이후 취업과 자립까지 도와주는 것을 뜻한다.'}, {'title': '<b>육아</b> - 두산백과 두피디아', 'link': 'https://terms.naver.com/entry.naver?docId=1133098&cid=40942&categoryId=32182', 'description': '어린아이를 기르는 일. 어린아이의 신체적 발육과 지적 교육, 정서의 건전한 발달을 위하여 노력하는 일을 말한다. 어린아이는 심신(心身)의 발육이라는 큰 특징을 갖는다. 어린아이의 몸과 마음의 성장을 장애하는...'}, {'title': '<b>육아</b> - 한국민족문화대백과', 'link': 'https://terms.naver.com/entry.naver?docId=539907&cid=46615&categoryId=46615', 'description': '아기를 가지면서부터 낳아 기르는 일을 가리키는 교육학용어. <b>육아</b>기의 구분은 시대와 지역에 따라 다를 수 있지만 대체로 초등학교 취학 이전까지로 보는 것이 적합하다. 태어나서 6, 7세까지의 이 시기는 모든...'}, {'title': '<b>육아</b> 영어로 - <b>육아</b> 영어 뜻', 'link': 'https://ko.ichacha.net/english/%EC%9C%A1%EC%95%84.html', 'description': '<b>육아</b> 영어로: <b>육아</b> [肉芽] (a) granulation. ~가 생기다 granulat.... 자세한 영어 의미 및 예문 보려면 클릭하십시오'}, {'title': '중앙<b>육아</b>종합지원센터', 'link': 'https://central.childcare.go.kr/', 'description': '영유아 <b>육아</b>지원 전문기관, 센터소개, 보육정책, 어린이집지원정보, 아동학대예방교육, 가정양육지원 등 제공.'}, {'title': '임신<b>육아</b>종합포털 아이사랑', 'link': 'https://www.childcare.go.kr/', 'description': '임신·출산·<b>육아</b>관련 정보 및 전문가 상담, 어린이집 이용(입소대기 신청, 보육료 결제 등) 서비스 제공.'}, {'title': '<b>육아</b> - 예스24', 'link': 'https://www.yes24.com/Product/Category/Display/001001001011', 'description': '삐뽀삐뽀 119 이유식 · #간편이유식 · #맘마먹자 · 최신 의학 지식에 근거한 과학적 이유식초보맘도 만들기 쉬운 만만한 이유식“전면 ... ; 삐뽀삐뽀 119 소아과 · [ 개정13판 ] · 1,490원 · #아이를키운다면한번쯤 · 세월이 흘러도 변하지 않는 ‘삐뽀삐뽀 119’의 가치,아기 키우 ... ; 똑게<b>육아</b> · 영유아 수면교육 · 로리(김준희) · 북로스트 · 1,110원 · #아이재우기 · #수면건강 ...'}, {'title': '일과 가정생활 &gt; <b>육아</b>휴직 및 <b>육아</b>기 근로시간 단축 &gt; <b>육아</b>기 근로시간 단축 &gt; <b>육아</b>기 근로시간 단축 신청 (본문....', 'link': 'http://easylaw.go.kr/CSP/CnpClsMain.laf?csmSeq=1380&ccfNo=2&cciNo=2&cnpClsNo=1', 'description': '<b>육아</b>, 근로시간, <b>육아</b>기 근로시간 단축, 근로시간 단축'}, {'title': '부평구<b>육아</b>종합지원센터', 'link': 'http://www.bpscc.or.kr/index.asp', 'description': '인천광역시 부평구 영성동로 18번길 51, 2층(삼산동) 위치, 행복한 <b>육아</b>실현 부평구<b>육아</b>종합지원센터가 함께합니다.'}, {'title': '<b>육아</b> | 국내도서 - 교보문고', 'link': 'https://product.kyobobook.co.kr/category/KOR/0704', 'description': '교보문고에서 제공하는 <b>육아</b> 분야의 베스트셀러, 신작, 추천도서를 확인해 보세요.'}]}


In [ ]:
df_naver = naver_api("kpop", 20)
df_naver


,Title,Link,Description
0,육아 - 나무위키,https://namu.wiki/w/%EC%9C%A1%EC%95%84,"育兒 아이를 기르는 것, 보통 미성년자인 자식을 키우는 것을 의미하며 좁게는 초등학..."
1,육아 - 두산백과 두피디아,https://terms.naver.com/entry.naver?docId=1133...,"어린아이를 기르는 일. 어린아이의 신체적 발육과 지적 교육, 정서의 건전한 발달을 ..."
2,육아 - 한국민족문화대백과,https://terms.naver.com/entry.naver?docId=5399...,아기를 가지면서부터 낳아 기르는 일을 가리키는 교육학용어. 육아기의 구분은 시대와 ...
3,육아 영어로 - 육아 영어 뜻,https://ko.ichacha.net/english/%EC%9C%A1%EC%95...,육아 영어로: 육아 [肉芽] (a) granulation. ~가 생기다 granul...
4,중앙육아종합지원센터,https://central.childcare.go.kr/,"영유아 육아지원 전문기관, 센터소개, 보육정책, 어린이집지원정보, 아동학대예방교육,..."
5,육아 - 예스24,https://www.yes24.com/Product/Category/Display...,삐뽀삐뽀 119 이유식 · #간편이유식 · #맘마먹자 · 최신 의학 지식에 근거한 ...
6,부평구육아종합지원센터,http://www.bpscc.or.kr/index.asp,"인천광역시 부평구 영성동로 18번길 51, 2층(삼산동) 위치, 행복한 육아실현 부..."
7,임신육아종합포털 아이사랑,https://www.childcare.go.kr/,"임신·출산·육아관련 정보 및 전문가 상담, 어린이집 이용(입소대기 신청, 보육료 결..."
8,일과 가정생활 &gt; 육아휴직 및 육아기 근로시간 단축 &gt; 육아기 근로시간 ...,http://easylaw.go.kr/CSP/CnpClsMain.laf?csmSeq...,"육아, 근로시간, 육아기 근로시간 단축, 근로시간 단축"
9,공동육아방(기관) 신청,https://www.jccic.or.kr/2018/index_re.php,서울형키즈카페면목4동점 ; 공동육아방 ; 놀이토리 프로그램 ; 영유아·부모교육신청


# Daum

In [115]:
import os, re, html, requests
import pandas as pd
from dotenv import load_dotenv


daum_search_api_endpoints = {
    "web": "웹 문서",
    "vclip": "동영상",
    "image": "이미지",
    "blog": "블로그",
    "book": "책",
    "cafe": "카페"
} # https://developers.kakao.com/docs/latest/ko/daum-search/dev-guide

load_dotenv()
Kakao_API_key = os.getenv("Kakao_API_key") # https://developers.kakao.com/

excluded_links = ["tistory", "kin", "youtube", "blog", "book", "news", "dcinside", "fmkorea", "ruliweb", "theqoo", "clien", "mlbpark", "instiz", "todayhumor"]


def remove_tags(text:str) -> str:
    tag_pattern = r'<(\/?[\w\s="\':^>]*)>'
    text = re.sub(tag_pattern, '', text).strip()
    text = html.unescape(text)
    return text


def daum_api(query:str, k:int, api:str="web", size:int=100, sort:str="accuracy"):
    """
    Parameters:
        query : 검색하고 싶은 검색어 
        k : 원하는 검색 결과 수 (1 ~ 300)
        api: 검색 api 종류('web', 'vclip', 'image', 'blog', 'book', 'cafe')
        size: 한 번에 표시할 검색 결과 개수
        sort: 검색 결과 정렬 방법 ("accuracy": 정확도순, "recency": 날짜순)

    Returns:
        df_daum : 검색 결과

        n | Title | Link | Description
        ---|---|---|---
        0 | 제목 | www.*.com | 설명
        . | ... | ... | ...
        k-1 | 제목 | www.*.com | 설명
    """

    results = []
    api = api if api in daum_search_api_endpoints else "web"
    url = f"https://dapi.kakao.com/v2/search/{api}"
    headers = {'Authorization': f'KakaoAK {Kakao_API_key}'}
    size = min(100, max(int(size), 10))
    sort = sort if sort in ["accuracy", "recency"] else "accuracy"
    params = {
        'sort': sort,
        'query' : query, 
        'page' : 0,
        'size' : size,
    }
    pages = k//size + (k%size > 0)
    k, count, flag = min(300, max(int(k), 1)), 0, False

    for page in range(1, pages + 10):
        params["page"] = page
        response = requests.get(url, params=params, headers=headers)

        if(status_code:= response.status_code) != 200:
            print(f"Error code: {status_code}")
            continue
        items = response.json().get("documents", [])

        for item in items:
            link = item.get('url', '')
            old = int(item.get('datetime', '2023')[:4] or '2023') < 2020
            if not link or (any(ex in link for ex in excluded_links) or old): continue
            title= item["title"]
            description = item.get('contents', 'no description')
            title, description = map(remove_tags, (title, description))
            results.append([title, link, description])
            count += 1
            if count >= k: flag = True; break
        if flag: break

    df_daum = pd.DataFrame(results, columns=['Title', 'Link', 'Description'])
    return df_daum 


In [69]:
{'documents': [{'contents': '모유 수유하는 것이 생각보다 쉽지 않다. 울면서 분유 수유로 돌아서는 사람이 대부분이다. F. 더드슨은 <b>육아</b>를 1년 365일 내내 쉬는 시간이 없는 직장을 갖는 일이라 표현했다. 20~30년간 철이 안 들었던 남녀를...', 'datetime': '2025-01-07T00:00:00.000+09:00', 'title': '<b>육아</b> - 나무위키', 'url': 'https://namu.wiki/w/%EC%9C%A1%EC%95%84'}, {'contents': '출산만 강요할 뿐 키우는 방법에 대해서는 함께 고민하지 않는 세상에서 부모라면 누구나 겪고 있는 <b>육아</b>전쟁백서를 다루는 드라마이다. 최근 일일 드라마 치고는 막장도가 그리 높은 편도 아니고. 남존여비...', 'datetime': '2024-11-24T00:00:00.000+09:00', 'title': '워킹 맘 <b>육아</b> 대디 - 나무위키', 'url': 'https://namu.wiki/w/%EC%9B%8C%ED%82%B9%20%EB%A7%98%20%EC%9C%A1%EC%95%84%20%EB%8C%80%EB%94%94'}, {'contents': 'https://n.news.naver.com/article/081/0003511336?sid=102  최근 2년 <b>육아</b>휴직 급여 부정수급 적발 규모 54억원 초과  <b>육아</b> 급여 휴직  저출산 문제 대응하기위한 일 가정 양립  지원책  사업주  친인척 위장...', 'datetime': '2025-01-16T17:50:01.000+09:00', 'title': '단독 2년간 줄줄 샌 ‘<b>육아</b>휴직 급여’ 54억 - 콘텐츠뷰', 'url': 'https://v.daum.net/v/KbZsnUVZng'}, {'contents': '밤새 <b>육아</b>에 지친 강아지가 있습니다. 졸린 눈,, 떡진 머리, 피곤한 모습까지 강아지들도 <b>육아</b>가 힘든가 봅니다.        출산 전에 뽀얀 피부와 부드러운 머릿결을 갖고 있던 모습이 정말 귀엽습니다...', 'datetime': '2025-01-16T11:00:02.000+09:00', 'title': '&#34;내 삶은 어디에...&#34; <b>육아</b>에 지친 강아지 - 콘텐츠뷰', 'url': 'https://v.daum.net/v/VtboPtBWbv'}, {'contents': 'https://www.donga.com/news/Economy/article/all/20250113/130847282/1 남성 <b>육아</b>휴직자 4만명 돌파 목전…2016년 대비 5배 넘게 증가 ⓒ News1 이재명 기자 지난해 1~11월 민간 부문의 전체 <b>육아</b>휴직자 10명 중...', 'datetime': '2025-01-13T10:35:59.000+09:00', 'title': '[🏛정책] ‘남성 <b>육아</b>휴직’ 1년새 20% 급증…작년 10명 중 3명 ‘아빠 <b>육아</b>’ [댓글수 ]', 'url': 'https://www.etoland.co.kr/bbs/board.php?bo_table=sisabbs01&wr_id=1491338'}, {'contents': '<b>육아</b> 휴직을 하고 싶어도 수입이 줄어드는 것 때문에 고민하는 분들 많잖아요. 내년부터는 이 <b>육아</b> 휴직 급여가 대폭 늘어나게 됐다고요? 내년부터 <b>육아</b>휴직을 내면 첫 석 달 동안은 매달 최대 250만 원을 받게...', 'datetime': '2024-12-18T09:59:41.000+09:00', 'title': '부부 <b>육아</b>휴직 급여 &#39;쑥&#39;, 연간 5,920만 원까지 인상...jpg', 'url': 'https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=9047963'}, {'contents': '미디어오늘 https://www.mediatoday.co.kr https://www.hani.co.kr/arti/society/labor/1176452.html <b>육아</b>휴직 양극화…10명 미만 중소기업 절반만 “모두 사용 가능” 김해정 기자 수정 2025-01-05 19:34 등록...', 'datetime': '2025-01-05T20:42:59.000+09:00', 'title': '[📰뉴스] <b>육아</b>휴직 양극화…10명 미만 중소기업 절반만 “모두 사용 가능” [댓글수 ]', 'url': 'https://www.etoland.co.kr/bbs/board.php?bo_table=sisabbs01&wr_id=1449899'}, {'contents': '든든한 버팀목 되다!&#34; [투어코리아=조성란 기자] 한화 김동선 부사장이 새해 첫 행보로 &#39;<b>육아</b> 동행&#39;에 나섰다. &#39;<b>육아</b> 동행 지원금&#39;은 직원 동행 프로젝트 일환으로 올해부터 시작된 제도로, 한화 유통·서비스...', 'datetime': '2025-01-06T21:53:26.000+09:00', 'title': '새해 첫 행보로 &#39;<b>육아</b> 동행&#39; 나선 한화 김동선 부사장…출산 가정에 1천만원 지원', 'url': 'http://www.tournews21.com/news/articleView.html?idxno=82884'}, {'contents': '동네 엄마들의 공동<b>육아</b> 커뮤니티 앱, ‘<b>육아</b>크루’ 운영사 다이노즈, ‘리얼 <b>육아</b> 콘텐츠’가 모여 있는 ‘맘블리’와 MOU 체결 이번 협약으로 맘블리 앰버서더들이 생산한 아티클의 확산을 위한 상호 협업 적극...', 'datetime': '2024-12-30T11:37:46.000+09:00', 'title': '동네 <b>육아</b> 커뮤니티앱 ‘<b>육아</b>크루’ 운영사 다이노즈, 리얼 <b>육아</b> 콘텐츠 플랫폼, ‘맘블리’와 MOU', 'url': 'https://www.startupn.kr/news/articleView.html?idxno=50060'}, {'contents': '1. 25년 <b>육아</b>휴직 관련 유급 기간이나 금액이 변경됬는데 언제부터 시행인가요? 2. 첫째 <b>육아</b> 휴직을 이미 1년 사용하였습니다. 그런데 25년부터 1년 6개월로 늘어났는데 추가로 6개월을 25년 지금 유급으로 사용...', 'datetime': '2025-01-15T13:39:59.000+09:00', 'title': '25년 <b>육아</b>휴직 관련 잘 아시는분 답변 부탁드려요', 'url': 'https://www.ppomppu.co.kr/zboard/view.php?id=baby&no=51743'}, {'contents': '사례로 알아볼 수 있도록 구성하였습니다. 아래에 항목을 선택하시면 내용을 보실 수 있습니다. <b>육아</b>휴직시 수당 <b>육아</b>휴직시 수당은 지급되나요? 1.평소 국토해양업무에 관심을 가져주신 점 깊이 감사드립니다. 2...', 'datetime': '2024-12-15T00:00:00.000+09:00', 'title': '영유아 보육 &gt; 영유아 보육 지원 &gt; <b>육아</b> 근로 지원 &gt; <b>육아</b>휴직 등 (국민신문고)', 'url': 'https://easylaw.go.kr/CSP/CnpClsMain.laf?csmSeq=626&ccfNo=3&cciNo=3&cnpClsNo=1&menuType=qna'}, {'contents': '2025년부터 대한민국의 <b>육아</b>휴직 급여 제도가 대폭 개선되었습니다. 가장 눈에 띄는 변화는 <b>육아</b>휴직 급여 상한액 인상으로, 기존 월 150만 원에서 최대 250만 원으로 상향 조정되었습니다. 이에 따라 1년간 <b>육아</b>...', 'datetime': '2025-01-11T22:20:00.000+09:00', 'title': '<b>육아</b>휴직급여 이번년도에 완전 바뀌었어요! | 네이트 판', 'url': 'https://pann.nate.com/talk/373791699'}, {'contents': '- 백석LH아파트에 자녀돌봄 품앗이가 가능한 ‘공동<b>육아</b>나눔터’ 본격 운영 천안시와 천안시건강가정지원센터(센터장 이은정)는 4일 ‘공동<b>육아</b>나눔터 10호점’(백석LH점)을 백석LH천년나무 1단지 아파트 내에...', 'datetime': '2024-12-20T17:45:59.000+09:00', 'title': '<b>육아</b>가 행복한 천안 ‘공동<b>육아</b>나눔터 10호점’ 개소', 'url': 'https://www.cheonan.go.kr/cop/bbs/BBSMSTR_000000000472/selectBoardArticle.do?nttId=B00000222984eg0kJ8ur4sp5'}, {'contents': '앞두고 있는 예비맘 입니다. 재직은 약 2년 9개월 정도, 원래 계획은 1월 말까지 근무 후 출산 휴가, <b>육아</b> 휴직을 계획하고 있었습니다. (팀장님께는 구두로 얘기한 상태) * 출산 예정일 3월 4일 근데 최근에...', 'datetime': '2024-12-21T09:48:00.000+09:00', 'title': '대기업 권고 사직, 출산 및 <b>육아</b>휴직 조언 | 네이트 판', 'url': 'https://pann.nate.com/talk/373678273'}, {'contents': '<b>육아</b>크루 서비스 소개 이미지 모바일앱 ‘엄마들의 동네 <b>육아</b>친구 찾기 - <b>육아</b>크루’를 운영하는 주식회사 다이노즈(대표 이가영, 이하 다이노즈)는 지난 9일 ‘포포포 매거진’을 운영하는 주식회사 포포포(대표...', 'datetime': '2024-12-24T14:46:28.000+09:00', 'title': '동네 <b>육아</b> 커뮤니티앱 ‘<b>육아</b>크루’ 운영사 다이노즈, ‘포포포 매거진’ 운영사 포포포와 MOU', 'url': 'https://www.startupn.kr/news/articleView.html?idxno=49987'}, {'contents': '<b>육아</b> 참여 낮은 한국 남성…&#34; 노벨상 수상자의 작심 발언 https://www.hankyung.com/article/2025010566011 클로디아 골딘 하버드대 교수 &#34;단순 보조금 확대는 대안 아냐&#34; 2023년 노벨경제학상 수상자인 클로디아...', 'datetime': '2025-01-06T19:57:57.000+09:00', 'title': '<b>육아</b> 참여 낮은 한국 남성…&#34; 노벨상 수상자의', 'url': 'https://gall.dcinside.com/mgallery/board/view/?id=centristpolitics&no=3069535'}, {'contents': '★ 공동<b>육아</b>방 □ 이용대상 : 만6세 이하 유아와 보호자 □ 사업목적 : 가정양육 부모의 고립<b>육아</b> 해소 및 지역 돌봄공동체 활성화 도모 □ 사업내용 - 자녀돌봄을 위한 안전한 놀이공간 제공 및 부모 참여형...', 'datetime': '2024-12-09T17:28:50.000+09:00', 'title': '공동<b>육아</b>방&amp;공동<b>육아</b>나눔터', 'url': 'https://news.seoul.go.kr/welfare/archives/509656'}, {'contents': '본 예산안도 209억원 소폭 줄었으나, 2024년 예산에 비해 1.6조원이 늘었다. 그 대부분이 모성보호·<b>육아</b>지원 사업이 증액된 것이다. 국회는 10일 본회의에서 총지출 673조3000억원 규모 내년도 예산안을 재석...', 'datetime': '2024-12-11T00:00:00.000+09:00', 'title': '내년 고용노동부 예산, 탄핵·감액 정국 속에 1.6조 증액...모성보호 <b>육아</b>휴직 중심 [The SIGNAL]', 'url': 'https://www.mimint.co.kr/bbs/view.asp?strBoardID=news&bbstype=S1N11&bidx=4669943'}, {'contents': '201 | 원자폭탄과 원자력 발전 …203 ◎ COLUMN ◎ 핵융합과 태양의 에너지원 …204 내 아이 성장발달 <b>육아</b>백과 도서명 : 내 아이 성장발달 <b>육아</b>백과 저자/출판사 : 손근형, 도도 쪽수 : 470쪽 출판일 : 2022-12...', 'datetime': '2024-12-18T10:33:59.000+09:00', 'title': '청소년을 위한 물리 이야기 또는 내 아이 성장발달 <b>육아</b>백과 &gt; NEW도서', 'url': 'https://www.kdgedu.co.kr/bbs/board.php?bo_table=product&wr_id=18766'}, {'contents': '안녕하세요. 19개월 아기와 살고있는 부부입니다. 결혼 후 싸운적이 없었는데 아기를 출산하고 <b>육아</b>하면서 다투는일이 일이 정말 많더군요... 정말 답답해서 글을 적습니다. 출산 후 아내가 <b>육아</b>휴직하여 아기를...', 'datetime': '2025-01-02T00:00:00.000+09:00', 'title': '집안일&amp;<b>육아</b> 분담으로 너무 많이 싸우는 3년차 부부 | 네이트 판', 'url': 'https://m.pann.nate.com/talk/373742479?currMenu=today&stndDt=20250103'}, {'contents': '여성 근로자가 모성을 보호하거나 근로자가 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위해 <b>육아</b>휴직을 신청하는 경우에는 이를 허용해야 하며, <b>육아</b>휴직의 기간은 한 자녀마다 1년 이내로 합니다...', 'datetime': '2024-12-15T00:00:00.000+09:00', 'title': '첫째 아이도 7살이기 때문에 <b>육아</b>휴직을 하고 싶습니다. <b>육아</b>휴직기간과 휴직기간 동안의 급여가 궁금...', 'url': 'https://easylaw.go.kr/CSP/OnhunqueansInfoRetrieve.laf?onhunqnaAstSeq=89&onhunqueSeq=4429'}, {'contents': '<b>육아</b>휴직 급여, 장려금 / 서울시 제공 2025년 <b>육아</b>휴직 급여가 월 최대 250만 원으로 인상되며, 사후지급금 제도가 폐지된다. 새로운 급여 체계에 따르면, 첫 3개월 동안 250만 원, 4~6개월은 200만 원, 7...', 'datetime': '2025-01-06T00:00:00.000+09:00', 'title': '2025년 <b>육아</b>휴직 급여부터 배우자 출산휴가 기간 20일로 확대', 'url': 'https://www.mimint.co.kr/bbs/view.asp?strBoardID=news&bbstype=S1N11&bidx=4694615'}, {'contents': '공무원임용령 개정안 내년 초 시행 <b>육아</b>휴직 전체 기간 승진 경력 인정 이미 쓴 <b>육아</b>휴직도 소급 적용 가능 AD 원본보기 아이콘 앞으로 자녀 수 상관없이 공무원 <b>육아</b>휴직 전체 기간이 승진을 위한 경력으로 인정...', 'datetime': '2024-12-31T17:36:18.000+09:00', 'title': '공무원 <b>육아</b>휴직 전체 기간 &#39;승진 근무경력&#39; 인정 - 아시아경제', 'url': 'https://www.asiae.co.kr/article/2024123111324191846'}, {'contents': '| KBS 250101 방송 등록일 2025.01.01 <b>육아</b> 휴직 자처! 재준 아빠의 <b>육아</b> 솜씨 보고 가실게요 (feat.카메라 러버 현조) Copyright ⓒ KBS. All rights reserved. 무단 전재, 재배포 및 이용(AI 학습 포함) 금지...', 'datetime': '2025-01-01T00:00:00.000+09:00', 'title': '<b>육아</b> 휴직 자처! 재준 아빠의 <b>육아</b> 솜씨 보고 가실게요(feat.카메라 러버 현조)', 'url': 'https://tv.zum.com/play/3010734?cm=front_tvclip&r=15&selectTab=highlight&thumb=1'}, {'contents': '▲ 인천 중구, 아빠 <b>육아</b>휴직 장려금 등 ‘저출생 극복’ 노력 박차 [국회일보] 인천시 중구는 <b>육아</b>휴직을 사용하는 남성에게 최대 6개월 동안 최대 300만원 장려금을 지원하는 ‘아빠 <b>육아</b>휴직 장려금’ 제도를...', 'datetime': '2024-12-17T13:19:27.000+09:00', 'title': '인천 중구, 아빠 <b>육아</b>휴직 장려금 등 ‘저출생 극복’ 노력 박차', 'url': 'http://www.assemblynews.co.kr/news/articleView.html?idxno=1215126'}, {'contents': '팀인데 같은 부서에서 일하는 40대 아주머니 직원이랑 단 둘이 엘리베이터를 탔는데, 나한테 갑자기 <b>육아</b>는 잘 하고 계시냐고 물어보다가- 대뜸 혹시 아내분이 독박<b>육아</b> 하고 있는거 아니냐고 물어보길래, 독박...', 'datetime': '2025-01-03T09:38:00.000+09:00', 'title': '독박<b>육아</b>라는 말 싫어하는데, 어제 회사 직원한테 한 소리 들었다.', 'url': 'https://www.inven.co.kr/board/lostark/6271/1131383'}, {'contents': '밑에 여동생에게 독박<b>육아</b>라고 하지 말라고 하는 분 이야기 듣다보니 여동생에게 전혀 공감을 못하고 계신데 혹시 딸이 있으면 딸 입장에서 한 번 생각해보시라고 적어봅니다. 독박<b>육아</b>라고 우울증 걸리는 대부분...', 'datetime': '2024-11-17T19:28:17.000+09:00', 'title': '독박<b>육아</b>라고 우울증 걸리는 이유 ::: 82cook.com 자유게시판', 'url': 'https://www.82cook.com/entiz/read.php?bn=15&num=3915301'}, {'contents': '&lt;상황&gt; - 남성근로자가 <b>육아</b>휴직 사용 후 퇴사하기로 합의, 사직서 및 확약서 작성 - <b>육아</b>휴직 : 2023년 2월 1일 ~ 2025년 1월 31일 (2자녀) - 퇴사일 : 2025년 1월 31일 - 사직서 사유 기재 내역 : 본인은 <b>육아</b>...', 'datetime': '2025-01-15T18:28:39.000+09:00', 'title': '<b>육아</b>휴직 종료 후 퇴사 관련 문의', 'url': 'https://www.nodong.kr:49336/qna/2420530'}, {'contents': '남양주시<b>육아</b>종합지원센터는 지난 13일 올 한 해 동안 추진한 주요 사업을 돌아보고 성과를 공유하기 위한 사업보고회를 개최했다. 이번 보고회에는 홍지선 남양주시 부시장을 비롯해 김대진 경복대학교 산학...', 'datetime': '2024-12-16T12:19:00.000+09:00', 'title': '남양주시<b>육아</b>종합지원센터, ‘함께 웃는 행복 <b>육아</b>’를 위한 사업보고회 성황리 개최', 'url': 'http://www.ddmnews.co.kr/news_gisa/gisa_view.htm?gisa_category=01000000&gisa_idx=482781'}, {'contents': '/TV조선 방송화면 캡처 <b>육아</b>휴직과 유연근로제 등 일·가정 양립을 위한 제도들의 도입과 활용이 꾸준히 늘고 있지만, 대기업과 중소기업간 격차는 여전히 큰 것으로 나타났다. 5일 한국여성정책연구원이 고용...', 'datetime': '2025-01-05T15:34:00.000+09:00', 'title': '10명 중 2명 &#34;<b>육아</b>휴직 못 써&#34;…중소기업은 &#39;그림의 떡&#39;', 'url': 'https://news.tvchosun.com/site/data/html_dir/2025/01/05/2025010590087.html'}, {'contents': '업무 성과 제고는 물론 사회에도 긍정적 기여할 것” 한화호텔앤드리조트가 새해부터 출산 가정에 ‘<b>육아</b> 동행 지원금 1000만원’을 지급한다. 일과 가정의 양립을 돕기 위한 ‘직원 동행 프로젝트’의 일환으로...', 'datetime': '2024-12-02T13:02:40.000+09:00', 'title': '직원 ‘<b>육아</b> 동행’ 나선 한화 유통·서비스 부문 새해부터 출산 가정에 ‘<b>육아</b> 동행 지원금’ 1000만원...', 'url': 'http://www.hotelavia.net/news/articleView.html?idxno=2958'}, {'contents': '통계청 &#39;2023년 <b>육아</b>휴직통계&#39; 출생아 줄며 육휴 대상자도 적어져 인식 개선에 사용률은 지속 상승세 이미지 확대보기 게티이미지뱅크 꾸준히 늘어왔던 연간 <b>육아</b>휴직자 규모가 지난해 처음 줄었다. <b>육아</b>휴직에...', 'datetime': '2024-12-18T16:06:00.000+09:00', 'title': '꾸준히 늘던 <b>육아</b>휴직자 규모 지난해 처음 꺾였다…이유는? | 한국일보', 'url': 'https://www.hankookilbo.com/News/Read/A2024121710580004355'}, {'contents': '앞으로 자녀당 최대 3년까지 쓸 수 있는 공무원 <b>육아</b>휴직의 전체 기간을 승진경력으로 인정한다. 근무 지역이나 기관을 정해서 채용된 구분 모집자도 출산이나 양육을 위한 사유라면 필수보직기간이 지나지...', 'datetime': '2024-12-31T00:00:00.000+09:00', 'title': '(인사혁신기획과) 공무원 <b>육아</b>휴직 전 기간, 승진경력 인정？', 'url': 'https://m.korea.kr/briefing/pressReleaseView.do?newsId=156668524'}, {'contents': '[싱가포르 입법 동향] 싱가포르, <b>육아</b>휴직 추가 부여에 관한 법안 의회 통과 싱가포르 의회는 유급 <b>육아</b>휴직 추가 부여 등을 포함한 「2001 아동발달공동저축법」 개정안을 2024년 11월 13일 통과시켰다. 동...', 'datetime': '2024-12-19T00:00:00.000+09:00', 'title': '싱가포르, <b>육아</b>휴직 추가 부여에 관한 법안 의회 통과', 'url': 'https://world.moleg.go.kr/web/dta/lgslTrendReadPage.do?CTS_SEQ=54322&AST_SEQ=279'}, {'contents': '요즘은 남편분들 <b>육아</b>휴직도 많이들 한다지만 그래도 참여를 잘 못하고 있잖아요. 무려 공영방송에서 제일 먼저 <b>육아</b>휴직하겠다고 용기내서 신청하는게 쉽지 않았을텐데... 대단한 것 같아요. <b>육아</b>게시판 오유님...', 'datetime': '2024-12-20T11:42:31.000+09:00', 'title': 'KBS 남자 아나운서 중 최초 <b>육아</b>휴직이라니 대단하네요', 'url': 'https://www.todayhumor.co.kr/board/view.php?table=baby&no=25715&s_no=25715&page=1'}, {'contents': '[아무튼, 주말] 요즘 <b>육아</b> 핫콘텐츠 애와 개 함께 키우기 &lt;이미지를 클릭하시면 크게 보실 수 있습니다&gt; 장밋빛 볼이 통통한 아기가 꼼지락대다 나비잠에 빠져든다. 이를 지켜보던 갈색 털에 까만 눈망울의...', 'datetime': '2024-11-27T00:00:00.000+09:00', 'title': '귀여움 두 배, 행복도 두 배? ‘애개 <b>육아</b>’의 현실은…', 'url': 'https://hub.zum.com/chosun/%EA%B7%80%EC%97%AC%EC%9B%80-%EB%91%90-%EB%B0%B0-%ED%96%89%EB%B3%B5%EB%8F%84-%EB%91%90-%EB%B0%B0-%EC%95%A0%EA%B0%9C-%EC%9C%A1%EC%95%84%EC%9D%98-%ED%98%84%EC%8B%A4%EC%9D%80-109942'}, {'contents': '애낳은 후 <b>육아</b> 현실 : 어떻게 이 똥 오줌 침들을 다 치울 수 있을까? 어떻게 이 짐승을 사람 만들 수 있을까?', 'datetime': '2025-01-05T13:59:00.000+09:00', 'title': '애낳기전 <b>육아</b> 상상 : 어떻게 애를 잘 가르치고 키울 수 있을까? - 자동차 - 에펨코리아', 'url': 'https://www.fmkorea.com/7891470637'}, {'contents': '<b>육아</b>전문 인플루언서 ‘쏘맘’ 문소영 씨 “빛이라곤 없는 어두운 터널 한가운데를 혼자 걷고 있는 것 같았어요. 언제 끝나는지 알 수 없고 어쩌면 영원히 빠져나갈 수 없는 터널인 것만 같았죠.” 문소영 씨는...', 'datetime': '2024-11-28T00:00:00.000+09:00', 'title': '‘<b>육아</b>우울증’ 경단녀서 SNS <b>육아</b> 전문가로 “<b>육아</b>는 도전이자 성장 엄마도 행복해질 수 있어” 작성일...', 'url': 'https://gonggamadmin.korea.kr/newsContentView.es?mid=a10205000000&section_id=NCCD_PUBLISH&content=NC002&code_cd=0105000000&nPage=1&b_list=9&news_id=416d8b18-d078-4901-9506-43a9be625201'}, {'contents': '“100인의 아빠단 올해도 아빠<b>육아</b> 참 잘했어요” - <b>육아</b>고수(멘토) 아빠단 감사장 수여 및 활동 소감 발표 등 14기 해단식 개최- 보건복지부(장관 조규홍)와 인구보건복지협회(회장 이삼식)는 11월 30일(토...', 'datetime': '2024-11-30T00:00:00.000+09:00', 'title': '“100인의 아빠단 올해도 아빠<b>육아</b> 참 잘했어요”', 'url': 'https://m.korea.kr/briefing/pressReleaseView.do?newsId=156663323'}, {'contents': '제주특별자치도는 일·가정 양립 지원 강화를 위해 확대 개편한 정부의 출산·<b>육아</b> 지원제도를 2025년 1월 1일부터 시행했다고 밝혔다. 우선 <b>육아</b>휴직 급여가 대폭 인상됐다. 첫 3개월 동안은 통상임금의 100%를...', 'datetime': '2025-01-14T22:26:22.000+09:00', 'title': '2025년 출산· <b>육아</b> 지원제도 확대 시행', 'url': 'http://www.samdanews.com/news/articleView.html?idxno=45877'}, {'contents': '<b>육아</b>휴직·유연근로 大-中企 격차 심각…18%는 &#39;육휴 사용불가&#39;[연합뉴스] <b>육아</b>휴직과 유연근로제 등 일·가정 양립을 위한 제도들의 도입과 활용이 꾸준히 늘고 있지만, 대-중소기업 간 격차가 여전히 심각...', 'datetime': '2025-01-05T16:15:46.000+09:00', 'title': '&#34;<b>육아</b>휴직·유연근로 大-中企 격차 심각…&#34; 18%는 육휴 사용불가', 'url': 'https://www.nanamcom.co.kr/news/articleView.html?idxno=12941'}, {'contents': '국공립어린이집 4개소 위·수탁 협약 체결 전북 정읍시가 2025년부터 <b>육아</b>수당을 기존 10만원에서 20만원으로 확대 지급하며 고물가 시대에 양육 부담 경감에 나선다고 27일 밝혔다. 시는 올해 <b>육아</b>수당 지급...', 'datetime': '2024-12-27T19:11:17.000+09:00', 'title': '정읍시, <b>육아</b>수당 월 10만원 → 20만원 확대', 'url': 'https://www.ezday.co.kr/bbs/view_board.html?q_sq_board=8886612'}, {'contents': '2015년 1.24명에서 8년 연속 하락해 지난해 사상 최저인 0.72명까지 내려왔다. 한국 정부가 출산·<b>육아</b> 관련 보조금을 확대하는 것과 관련해 골딘 교수는 &#34;유럽에서도 잘 작동하지 않고 있다&#34;며 출산율을 높이기...', 'datetime': '2025-01-05T05:00:00.000+09:00', 'title': '&#39;노벨상 수상자&#39; 골딘 한국 초저출산은 女 독박<b>육아</b> 탓[전미경제학회 2025]', 'url': 'https://www.ppomppu.co.kr/zboard/view.php?id=news_society&no=404748'}, {'contents': '내년 1월부터 근로자가 <b>육아</b>휴직을 12개월 사용하면 전체 급여액을 기존 1800만원에서 총 2310만원까지 지급하는 등 일·가정 양립을 위한 정부 지원이 확대된다. 또한 부모가 함께 생후 18개월 이내 사용할 수...', 'datetime': '2024-12-17T17:57:00.000+09:00', 'title': '내년부터 <b>육아</b>휴직 급여 월 최대 250만 원으로 확대 - 신문고뉴스', 'url': 'http://www.shinmoongo.net/sub_read.html?uid=171710&section=sc2&section2=경제'}, {'contents': '를 향한 애틋한 고백이 담긴 편지의 주인공은 누구일지 궁금증을 끌어올린다. 또한 코드쿤스트는 엄마의 <b>육아</b> 일기가 담긴 공책도 발견한다. 그는 자신이 태어난 날부터 약 1년간 기록된 <b>육아</b> 일기 속 생생한...', 'datetime': '2025-01-16T13:55:50.000+09:00', 'title': '코드쿤스트, <b>육아</b> 난이도 최강 모먼트..母<b>육아</b>일기 발견 &#39;울컥&#39; [나혼산]', 'url': 'https://www.starnewskorea.com/stview.php?no=2025011613384419122'}, {'contents': '시도별 <b>육아</b>휴직자 수(통계청 제공) [경남뉴스 | 권연수 기자] 지난해 <b>육아</b>휴직자 수가 사상 처음으로 전년 대비 감소했다. 출생아 수가 줄면서 <b>육아</b>휴직을 할 부모도 줄어든 영향이다. 지난해 직장인들이 고물가...', 'datetime': '2024-12-18T18:39:43.000+09:00', 'title': '‘저출생’여파 작년 <b>육아</b>휴직자 첫 감소....사용률은 역대 최대', 'url': 'https://www.gnnews24.kr/news/articleView.html?idxno=26291'}, {'contents': '▲ 고용노동부(사진=PEDIEN) [국회일보] 2025년 1월 1일부터 <b>육아</b>휴직 급여가 대폭 인상되며 사후지급방식이 폐지된다. 이에 따라 <b>육아</b>휴직 기간 동안 급여가 상향 조정되고 지급 방식도 개선된다. 특히 <b>육아</b>휴직...', 'datetime': '2024-12-31T12:35:32.000+09:00', 'title': '2025년부터 <b>육아</b>휴직 급여 인상 및 사후지급방식 폐지…한부모 근로자 지원 강화', 'url': 'http://www.assemblynews.co.kr/news/articleView.html?idxno=1222810'}, {'contents': '첨부된 문서 결재문서본문.hwpx (46.19 KB) PDF다운로드 원문다운로드 임기제공무원 인사발령(출산휴가<b>육아</b>휴직 연계).hwpx 비공개 문서 한강사업총괄부 한강개발기획과 업무분장.hwpx 비공개 문서 <b>육아</b>휴직에...', 'datetime': '2024-12-24T00:00:00.000+09:00', 'title': '<b>육아</b>휴직에 따른 업무대행수당 지급 요청', 'url': 'https://opengov.seoul.go.kr/sanction/:/opengov.seoul.go.kr/sanction/32522874'}, {'contents': '망고스틴 찾아보기 토핑(시기별) 토핑 이유식(시기별) 토핑(가나다순) 엄마와 아이가 함께 성장하는 균형<b>육아</b> 도서명 : 엄마와 아이가 함께 성장하는 균형<b>육아</b> 저자/출판사 : 고정희, 청년정신 쪽수 : 288...', 'datetime': '2024-12-18T10:31:59.000+09:00', 'title': '뿐이 토핑 이유식 또는 엄마와 아이가 함께 성장하는 균형<b>육아</b> &gt; NEW도서', 'url': 'https://www.kdgedu.co.kr/bbs/board.php?bo_table=product&wr_id=14347'}, {'contents': '- 12월 7일, 인천 100인 아빠단 6기 해단식 및 문화행사 성료 - <b>육아</b>에 서툰 아빠들, ‘인천 100인의 아빠단’으로 자신감 얻다 - 인천시, 아빠들의 <b>육아</b> 실천을 지원하는‘인천아빠 <b>육아</b>천사단’도 운영 인천...', 'datetime': '2024-12-11T12:31:59.000+09:00', 'title': '인천시, 아빠들의 <b>육아</b> 참여로 사회적 인식 변화 이끈다', 'url': 'http://koreaarttv.com/detail.php?number=97628&thread=23r02'}, {'contents': '10월 1일(금)부터 원활한 공동<b>육아</b>방 이용을 위해 예약사이트를 변경 합니다. 예약사이트 바로가기 시설안내 공동<b>육아</b>방 시설 안내 표 : 구분,위치,운영현황(운영시간,이용인원),연락처,비고 항목으로 구성 구분...', 'datetime': '2025-01-06T00:00:00.000+09:00', 'title': '공동<b>육아</b>방', 'url': 'https://www.jungnang.seoul.kr/portal/main/contents.do?menuNo=201066'}, {'contents': '통계청, ‘2023년 <b>육아</b>휴직통계 결과’ <b>육아</b>휴직자 19만5986명…전년比 3.0%↓ 아빠 7.5%·엄마 1.4%↓…사용률은↑ “기저효과 작용…인구구조 변화 영향” ⓒ뉴시스 저출산의 영향으로 지난해 <b>육아</b>휴직자 수가...', 'datetime': '2024-12-18T15:53:59.000+09:00', 'title': '저출산에 작년 <b>육아</b>휴직 부모 첫 감소…엄마가 아빠 ‘2.9배’', 'url': 'https://bizn.donga.com/East/3/all/20241218/130671414/1'}, {'contents': '© 의양신문 [의양신문=장재원기자]양주시(시장 강수현)가 최근 ‘양주시<b>육아</b>종합지원센터(센터장 김광이)’에서 ‘2024년 <b>육아</b>쉼표 사업’을 2년 연속 성공적으로 수행하여 ‘교육부 장관상’을 수상했다고 27일...', 'datetime': '2025-01-15T00:00:00.000+09:00', 'title': '양주시, ‘양주시<b>육아</b>종합지원센터’ 지역 기반 부모 양육지원 ‘힘써’ⵈ ‘교육부 장관상’ 수상', 'url': 'http://www.uynews.net/sub_read.html?uid=175619&section=sc32&section2=경제/사회'}, {'contents': '박수홍이 현실 <b>육아</b>에 넋을 놓기 일보 직전이다. 오늘 11일(수) 방송되는 KBS2 ‘슈퍼맨이 돌아왔다’에서는 ‘내 마음속 1등은 늘 너야~’ 편으로 3MC 박수홍, 최지우, 안영미와 슈퍼맨 김준호, 박수홍, 장동민...', 'datetime': '2024-12-11T16:11:17.000+09:00', 'title': '생후 50일 딸 현실 <b>육아</b>에 초췌해진 박수홍 (슈돌)', 'url': 'https://www.ezday.co.kr/bbs/view_board.html?q_sq_board=8884556'}, {'contents': '/사진=&#39;슈퍼맨이 돌아왔다&#39; 방송화면 캡쳐 &#39;슈퍼맨이 돌아왔다&#39;에서 강재준이 <b>육아</b>에 몰두한 근황을 공개했다. 1일 방송된 KBS 2TV &#39;슈퍼맨이 돌아왔다&#39; 555회에서는는 &#39;푸른 뱀의 해엔 행복 가득, 사랑 가득&#39; 편...', 'datetime': '2025-01-01T20:59:59.000+09:00', 'title': '&#39;이은형♥&#39; 강재준, 수유→배변 기록까지..&#34;<b>육아</b> 위해 휴직&#34;[슈돌]', 'url': 'https://www.starnewskorea.com/stview.php?no=2025010120400079258'}, {'contents': '□ &#39;23년 전체 <b>육아</b>휴직자*는 19만 5,986명으로 전년대비 3.0% 감소하였음 * 임신 중이거나 8세 또는 초등학교 2학년 이하의 자녀를 대상으로 <b>육아</b>휴직을 시작한 자 ㅇ ‘23년 전체 <b>육아</b>휴직자 중 모(母)는 74.3%...', 'datetime': '2024-12-18T00:00:00.000+09:00', 'title': '2023년 <b>육아</b>휴직통계 결과(잠정) | 전체 | 보도자료 | 새소식 : 통계청', 'url': 'https://kostat.go.kr/board.es?mid=a10301010000&bid=11814&list_no=434241&act=view&mainXml=Y'}, {'contents': '내년부터 <b>육아</b> 지원 3법 개정안이 시행됨에 따라 현재 월 최대 150만원인 <b>육아</b>휴직 급여가 월 최대 250만원으로 오르고, <b>육아</b>휴직 대체인력 지원금도 확대된다. 또 <b>육아</b>휴직 기간은 최대 1년 6개월까지 늘어나고...', 'datetime': '2024-12-30T10:47:59.000+09:00', 'title': '“새로운 <b>육아</b>제도 어떻게 바뀌나?”…QnA로 살펴보기-조선에듀', 'url': 'https://edu.chosun.com/site/data/html_dir/2024/12/30/2024123080071.html?main_top'}, {'contents': '신한금융그룹, 대‧중소기업‧농어업협력재단(대중소상생재단), 그리고 5개 자치단체가 힘을 모아 <b>육아</b>휴직 등에 따른 대체인력 1인당 연간 최대 1840만원을 지원한다. 먼저, 대체인력 지원 대상은 출산전‧후...', 'datetime': '2025-01-05T20:59:02.000+09:00', 'title': '근로자 <b>육아</b>휴직 시 혜택 늘어나 - 트래블데일리', 'url': 'http://www.traveldaily.co.kr/news/articleView.html?idxno=57167'}, {'contents': '<b>육아</b>휴직 기간 1년→1년6개월, 배우자 출산 휴가 10일→20일 ‘상습 임금체불 근절법’ 시행…상습 임금 체불 사업주 제재 강화 [123RF] [헤럴드경제=이태형 기자]2025년부터 <b>육아</b>휴직 급여 인상, <b>육아</b>휴직 기간...', 'datetime': '2024-12-31T10:28:06.000+09:00', 'title': '<b>육아</b>휴직 급여 월 상한 150만원에서 250만원으로 인상[새해 달라지는 것]', 'url': 'https://mbiz.heraldcorp.com/article/10333134'}, {'contents': '한 엄마와 아이가 서울 시내 한 직장어린이집으로 향하고 있다. 뉴스1 고용노동부는 17일 국무회의에서 <b>육아</b>휴직 급여 상한액을 월 250만 원으로 올리고, 출산휴가와 <b>육아</b>휴직을 함께 신청할 수 있게 하는 등의...', 'datetime': '2024-12-17T10:25:37.000+09:00', 'title': '내년부터 <b>육아</b>휴직 급여 최고 월 250만 원 | 한국일보', 'url': 'https://www.hankookilbo.com/News/Read/A2024121710130004814'}, {'contents': '<b>육아</b>휴직을 4개월 만에 마무리하게 되었어요. 애초에 1년을 계획했었는데 말이죠. 하핫. 꿈같은 시간들이었어요. 아이와 둘이 여행을 다니고, 에세이도 여러 편 쓰고, 밤에는 아내 논문을 봐주었어요. 제 욕심...', 'datetime': '2025-01-03T10:51:33.000+09:00', 'title': '<b>육아</b>휴직의 최후 : 클리앙', 'url': 'https://www.clien.net/service/board/cm_baby/18889190'}, {'contents': '<b>육아</b>휴직 13년 만에 첫 감소…저출생 영향 통계청, 2023년 <b>육아</b>휴직통계 안태호기자 수정 2024-12-18 12:16 등록 2024-12-18 12:00 기사를 읽어드립니다 Your browser does not support theaudio element. 게티...', 'datetime': '2024-12-18T12:00:00.000+09:00', 'title': '<b>육아</b>휴직 13년 만에 첫 감소…저출생 영향', 'url': 'https://www.hani.co.kr/arti/economy/economy_general/1173735.html'}, {'contents': '앞으로는 <b>육아</b>휴직 대신 단축근무를 선택할 경우 급여 외에 받을 수 있는 단축급여가 통상임금의 40%에서 60%로 확대되고, 단축근무를 최대 2년까지 할 수 있다. 하루 8시간 근무로 월 소득 100만원을 받는...', 'datetime': '2025-01-09T00:00:00.000+09:00', 'title': '<b>육아</b>위한 단축근무 시 임금의 60% 지원', 'url': 'http://xn--vg1b002a5sdzqo.kr/read.php3?no=22706&read_temp=&section=1'}, {'contents': '3회째 맞은 키즈랜드 토크 콘서트, 서울과 수원서 개최 ‘국민 <b>육아</b>멘토’ 오은영 박사가 <b>육아</b> 고민 해결 나서 KT 고객 무료FH 참여… 15일까지 KT 홈페이지로 참가 (사진=KT) [화이트페이퍼=이승섭 기자] KT...', 'datetime': '2024-09-30T13:34:46.000+09:00', 'title': 'KT, ‘<b>육아</b> 고민 나누는 키즈랜드 토크 콘서트’ 참가자 모집', 'url': 'https://www.whitepaper.co.kr/news/articleView.html?idxno=238577'}, {'contents': '내년 4조원 필요, 부담 가중 내년 <b>육아</b>휴직급여 예산 대폭 증가 21일 기획재정부 등에 따르면, 내년 <b>육아</b>휴직급여 예산의 정부안은 4조225억원이다. 올해 2조4968억원보다 약 1조5000억원 늘었다. 이는 2022...', 'datetime': '2024-10-22T15:22:00.000+09:00', 'title': '‘<b>육아</b>휴직급여’ 메우려… 고용보험 또 오르나', 'url': 'https://theqoo.net/square/3453212904'}, {'contents': '2030년까지 합계출산율 1명대를 달성하겠다는 목표로 신혼집 마련 지원부타 난임 극복, 돌봄 강화, <b>육아</b>휴직 개선까지 보다 폭넓은 대책들을 추진한다는 방침이다. 먼저 정부는 내년부터 출산 가구를 대상으로...', 'datetime': '2024-12-28T10:51:20.000+09:00', 'title': '“<b>육아</b>휴직급여는 높이고 주택은 공급하고”... 2025년 저출생 대책 총정리', 'url': 'http://www.hg-times.com/news/articleView.html?idxno=268262'}, {'contents': '수석 입사하고 본사 핵심 부서에서 일하셨던 분이 있어요. 그런데 00년대 초에 <b>육아</b>휴직 썼다고 찍혔었다네요. 그 부서에선 <b>육아</b>휴직 쓴 게 처음이었대요. 복직 후 비인기 부서로 발령나고 동기들한테 승진에서...', 'datetime': '2024-09-19T20:01:30.000+09:00', 'title': '요즘도 <b>육아</b>휴직 썼다고 눈치 주는 직장 있나요? ::: 82cook.com 자유게시판', 'url': 'https://www.82cook.com/entiz/read.php?bn=15&num=3886183'}, {'contents': '않다&#34;며 &#34;왜 남성공무원은 제외되는지 의문&#34;이라고 지적했다.이어 &#34;제도를 시행하려면 남성공무원도 <b>육아</b> 등에 적극적으로 참여할 기회는 보장받아야 한다&#34;며 &#34;제도에 보완이 필요하다&#34;고 덧붙였다. 또 자신을...', 'datetime': '2024-11-23T14:18:00.000+09:00', 'title': '여성은 당직 제외…&#34;남성 공무원은 <b>육아</b> 안하냐?&#34; 역차별 반발', 'url': 'https://theqoo.net/square/3497880795'}, {'contents': '내용 정보를 제공합니다.) 작성자 : 공보관 작성일 : 2024/11/29 조회 : 첨부파일 수원시 <b>육아</b>종합지원센터·동부<b>육아</b>종합지원센터, 조직문화 공모전 수상 수원시동부<b>육아</b>종합지원센터 최우수상, 수원시<b>육아</b>종합...', 'datetime': '2024-11-29T00:00:00.000+09:00', 'title': '수원시 <b>육아</b>종합지원센터·동부<b>육아</b>종합지원센터, 조직문화 공모전 수상', 'url': 'http://www.suwon.go.kr/web/board/BD_board.view.do?bbsCd=1043&seq=20241129100820350'}, {'contents': '24년 출산휴가 3+<b>육아</b>휴직3개월 사용하고 24/10/02일 자로 복직 하였습니다. 기존 하던 일은 해외영업으로 4년 차 입니다. 복직 후 얼마 지나지 않아 타 부서 (영업지원팀)로 발령 내겠다는 얘기를 들었고 이에...', 'datetime': '2025-01-02T22:49:23.000+09:00', 'title': '<b>육아</b>휴직 복귀 후 부서강제이동', 'url': 'https://www.nodong.kr/qna/2419125'}, {'contents': '아빠랑 다독다독 크리스마스’ 프로그램을 진행했다. 【백운신문/양평방송】이정아기자 = 경기 양평군 <b>육아</b>종합지원센터(센터장 조연경)는 12월 놀이지도사 사업으로 아빠가 읽어주고 놀아주는 ‘아빠랑 다독다독...', 'datetime': '2024-12-24T18:05:19.000+09:00', 'title': '양평군<b>육아</b>종합지원센터, ‘아빠랑 크리스마스’ &#34;행복<b>육아</b> 축복&#34;', 'url': 'http://www.ypnews.kr/news/articleView.html?idxno=53129'}, {'contents': '내년부터 <b>육아</b>휴직 급여 상한액이 월 250만원으로 현재보다 100만원 오른다. 출산휴가와 <b>육아</b>휴직은 함께 신청할 수 있고, <b>육아</b>휴직을 신청했는데 사업주가 2주 내 허용하지 않을 경우 신청한 대로 휴직할 수...', 'datetime': '2024-12-17T14:00:00.000+09:00', 'title': '내년부터 <b>육아</b>휴직 급여 최고 월 250만원으로 인상', 'url': 'https://news.tvchosun.com/site/data/html_dir/2024/12/17/2024121790151.html'}, {'contents': '돌아왔다&#39; 캡처 (서울=뉴스1) 박하나 기자 = &#39;슈퍼맨이 돌아왔다&#39; 코미디언 강재준이 아내를 위해 평생 <b>육아</b>를 맡겠다고 밝혔다. 1일 오후에 방송된 KBS2TV 예능 프로그램 &#39;슈퍼맨이 돌아왔다&#39;(이하 &#39;슈돌...', 'datetime': '2025-01-01T21:07:01.000+09:00', 'title': '강재준 &#34;아내 이은형 위해 평생 <b>육아</b>&#34;…&#39;슈돌&#39; 출격', 'url': 'https://www.moneys.co.kr/article/2025010121078012649'}, {'contents': '지금은 복직한지 1년차고 회사는 총 10년다녔고 60세까지 정년이 보장되어있는 회사야 업무강도도 높지않고 <b>육아</b>시간써서 4시에 퇴근이라직업 만족도가 정말 높은데 집에서는 커피를 시켜도 다 먹지도 못하고...', 'datetime': '2025-01-14T00:00:00.000+09:00', 'title': '<b>육아</b>보다 회사가 너무 좋다.. | 네이트 판', 'url': 'https://m.pann.nate.com/talk/373805785?currMenu=today&stndDt=20250115'}, {'contents': '(서울=뉴스1) 양혜림 디자이너 = 13일 고용노동부에 따르면 2024년 1~11월 남성 <b>육아</b>휴직자는 3만 9463명으로 전년 동월(3만 2932명) 대비 19.9% 증가했다. 전체 <b>육아</b>휴직자 중 남성 <b>육아</b>휴직자의 비율은 매년...', 'datetime': '2025-01-13T15:03:59.000+09:00', 'title': '[오늘의 그래픽] <b>육아</b>휴직 10명 중 3명 &#39;아빠&#39;…남성 사용률 1년새 20% 급증', 'url': 'https://www.news1.kr/photos/7082372'}, {'contents': '전라북도 공무원들이 올해부터는 업무공백의 부담을 느끼지 않고 자유롭게 <b>육아</b>휴직을 갈 수 있는 길이 열렸다. 전라북도는 1. 3일자로 2013년 공채 임용대기자 38명 중 5명을 <b>육아</b>휴직자 대체인력으로 배치하는...', 'datetime': '2024-12-24T00:00:00.000+09:00', 'title': '공무원 ‘맘(Mom)’ 편히 <b>육아</b>휴직 간다', 'url': 'http://xn--vg1b002a5sdzqo.kr/read.php3?no=22502&read_temp=&section=50'}, {'contents': '청년도약계좌·장병내일준비적금 정부지원금 확대 정부가 일과 가정의 양립 활성화를 위해 올해부터 <b>육아</b>휴직 급여와 기간을 확대한다. 월 최대 250만원을 1년 6개월에 걸쳐 전액 지급받을 수 있다. 기업의 출산...', 'datetime': '2025-01-05T23:53:20.000+09:00', 'title': '[새해 달라지는 우리생활]<b>육아</b>휴직 급여·기간 확대… 일·가정 양립 기여', 'url': 'https://insnews.co.kr/design_php/news_view.php?firstsec=1&secondsec=14&num=82130'}, {'contents': '고용노동부는 10월 16일, 판교 세븐벤처밸리 어린이집에서 일하는 부모들과 간담회를 개최하여 <b>육아</b>지원제도의 확대 방안을 논의했다. 이번 간담회는 지난 9월 26일 국회를 통과한 「<b>육아</b>지원 3법」의 내년 시행...', 'datetime': '2024-10-16T09:45:38.000+09:00', 'title': '<b>육아</b>휴직 급여 최대 250만 원까지 인상, <b>육아</b>휴직 1년 6개월, 이제 현실로!', 'url': 'https://www.eachj.co.kr/news/articleView.html?idxno=12317'}, {'contents': '국립부경대학교 산학협력단 직원(<b>육아</b>휴직대체) 채용 계획을 다음과 같이 공고하오니, 국립부경대학교 산학협력단 발전에 유능한 인재의 많은 지원 바랍니다. 1. 채용분야 및 인원 - 채용분야(담당 업무): <b>육아</b>...', 'datetime': '2025-01-07T00:01:00.000+09:00', 'title': '[채용 공고] 국립부경대학교 산학협력단 직원(<b>육아</b>휴직대체) 채용 공고', 'url': 'https://www.korea.kr/archive/recruitInfoView.do?dataId=340972&pWise=main&pWiseMain=Q3'}, {'contents': '<b>육아</b>를 하다 보면 부모들은 아이의 요구와 현실적인 한계 사이에서 끊임없이 고민하게 되죠. 특히 장난감은 그 고민의 중심에 있습니다. 아이들은 처음엔 반짝이는 눈으로 손에 장난감을 쥐었지만 금세 흥미를...', 'datetime': '2025-01-07T14:27:59.000+09:00', 'title': '<b>육아</b>의 쉼표, 반납할 수 없는 행복을 만들어준 천안의 특별한 놀이터 호두국의불당소녀 생활정보', 'url': 'https://www.cheonan.go.kr/prog/reporter/kor/sub05_05_01/allview.do?rotNo=6504'}, {'contents': '아빠 <b>육아</b> 골든벨 포스터 (사진=영등포구) 한강타임즈 백진아 기자 = 영등포구(구청장 최호권)는 오는 10월 26일, YDP 미래평생학습관에서 <b>육아</b> 최고수 아빠들을 가리는 ‘도전! 아빠 <b>육아</b> 골든벨’을 개최한다고...', 'datetime': '2024-09-24T10:01:00.000+09:00', 'title': '영등포 ‘<b>육아</b>의 신’ 가린다...영등포구, 아빠 <b>육아</b> 골든벨 개최', 'url': 'http://www.hg-times.com/news/articleView.html?idxno=265513'}, {'contents': '<b>육아</b>휴직급여 4년간 72억 샜다…&#34;가짜 육휴 쓰고 부정수급&#34; 스크롤 이동 상태바 <b>육아</b>휴직급여 4년간 72억 샜다…&#34;가짜 육휴 쓰고 부정수급&#34; 강득구 민주당 의원, 노동부 자료 분석 사업주와 근로자 공모 부정수급...', 'datetime': '2025-01-16T11:07:02.000+09:00', 'title': '<b>육아</b>휴직급여 4년간 72억 샜다…&#34;가짜 육휴 쓰고 부정수급&#34; &lt; 정치·사회 &lt; 기사본문', 'url': 'https://www.smartfn.co.kr/news/articleView.html?idxno=100914'}, {'contents': '임산부 지원 출산 지원 <b>육아</b>(다자녀) 지원 생활문화 지원 노인 지원 2025. 1. 기준 사업명, 사업내용(지원대상 및 지원수준), 문의처(전화번호) 순으로 구성된 <b>육아</b>(다자녀)지원 안내표 사업명 사업내용 (지원...', 'datetime': '2025-01-09T00:00:00.000+09:00', 'title': '<b>육아</b>(다자녀) 지원', 'url': 'https://www.haeundae.go.kr/index.do?menuCd=DOM_000000103001005003'}, {'contents': '의결 내년 지방공무원 임금이 3% 인상되고 저연차 공무원은 추가 인상률을 적용해 6.6%까지 오른다. <b>육아</b>휴직 시 경력 인정기간을 늘리고 지급 상한액을 확대하는 등 공무원의 <b>육아</b> 여건도 개선된다. 행정안전부...', 'datetime': '2024-12-31T17:30:00.000+09:00', 'title': '저연차 공무원 임금 최대 6.6% 인상…<b>육아</b>여건도 개선 - 아시아경제', 'url': 'http://www.asiae.co.kr/article/2024123114262680175'}, {'contents': '프로그램이 없는 경우 뷰어 프로그램을 설치하여 열람하시기 바랍니다. 한컴오피스 다운로드 바로가기 <b>육아</b>휴직 예정자 휴직기간 변경 요청 문서보기는 문서변환기에 의해 텍스트로 변환된 문서를 보여주며...', 'datetime': '2024-12-11T00:00:00.000+09:00', 'title': '<b>육아</b>휴직 예정자 휴직기간 변경 요청 &gt; 결재문서 &gt; 원문정보 &gt; 정보소통광장', 'url': 'https://opengov.seoul.go.kr/sanction/:/opengov.seoul.go.kr/sanction/:/opengov.seoul.go.kr/sanction/:/opengov.seoul.go.kr/sanction/32417620'}, {'contents': '최근 <b>육아</b>방식 중 어릴 적 부터 매운 거 먹이는 거 별로 안 좋다라는게 풍조 그래서 부모들은 매운 거 안 먹이고 아이들도 매운 건 안 먹게 됨 딱히 큰 문제는 없고 오히려 설득력있는 <b>육아</b>방식이지만 문제는...', 'datetime': '2025-01-14T12:33:05.000+09:00', 'title': '[유머] 최근 <b>육아</b>방식때문에 은근 고생하는 초등학생들', 'url': 'https://bbs.ruliweb.com/best/board/300143/read/69161215'}, {'contents': '가치자람 사회적협동조합 아빠<b>육아</b>문화연구소 김기탁 소장 ‘아내 카톡에 답장 잘하자’, ‘일 갔다 오면 아내에게 먼저 수고했다고 말하자’, ‘아이는 아빠가 씻기자’, ‘성과보다 중요한 것은 가족 자존감...', 'datetime': '2024-10-24T00:00:00.000+09:00', 'title': '초보아빠에서 <b>육아</b>멘토로 “최고의 <b>육아</b>는 함께 놀아주는 것 하루 10분만 시간내보세요!” 작성일...', 'url': 'https://gonggamadmin.korea.kr/newsContentView.es?mid=a10205000000&section_id=NCCD_PUBLISH&content=NC002&code_cd=0105000000&nPage=1&b_list=9&news_id=e8830a08-0b26-4fd2-891d-e696b26d712a'}, {'contents': '(사진=LG유플러스) [화이트페이퍼=최승우 기자] LG유플러스가 자사 <b>육아</b> 정보 제공 앱 ‘부모나라’에서 실시간 <b>육아</b> 상담 서비스 ‘ixi(익시) <b>육아</b> 매니저’를 출시했다고 22일 밝혔다. 익시는 부모가 실시간...', 'datetime': '2024-07-22T15:04:46.000+09:00', 'title': 'LGU+, 실시간 AI <b>육아</b>상담 서비스 ‘익시 <b>육아</b> 매니저’ 출시', 'url': 'https://www.whitepaper.co.kr/news/articleView.html?idxno=236326'}, {'contents': '<b>육아</b>휴직쓰고 그만두고싶은데 퇴사를 목적으로 <b>육아</b>휴직쓰면 회사에서 정당하게 거절할 수 있는거야?? 아니면, 나는 퇴사를목적으로 써도 회사에서는 모르면 상관없는거야?? 또, 퇴사를 목적으로했다고 <b>육아</b>휴직...', 'datetime': '2024-11-24T01:12:00.000+09:00', 'title': '<b>육아</b>휴직 써본 형냐들있어 혹시?', 'url': 'https://www.inven.co.kr/board/fifaonline4/3146/8312010'}, {'contents': '상담원으로 일하고 있는 미선씨. 아이를 키우고 있는 워킹맘으로서 내년 2025년 2월부터 적용되는 <b>육아</b>지원 3법을 자세하게 알려준다고 하는데요. 미선씨의 하루를 보며 달라지는 <b>육아</b>지원 3법에 대한 자세한...', 'datetime': '2024-12-24T00:00:00.000+09:00', 'title': '<b>육아</b>지원 3법, 그래서 그게 뭔데? 고용노동부 직원이 직접 알려준다!', 'url': 'https://www.korea.kr/multi/mediaNewsView.do?newsId=148937824'}, {'contents': '주어 윤상abn과 앤톤 연말 시상식 직캠 최초 진짜<b>육아</b>하는 장면이 찍힌게 너무웃겨미침 pic.twitter.com/Xosi26YG0K— 𝐘 (@formyitboy) January 2, 2024', 'datetime': '2025-01-05T15:47:17.000+09:00', 'title': '[잡담] 연말 시상식 직캠 최초로 찐 <b>육아</b>장면이 찍혔다는 말이 너무 웃겨 - 인스티즈(instiz) 연예...', 'url': 'https://www.instiz.net/name_enter/94679035'}, {'contents': '2025년 <b>육아</b>휴직 개정 내용 (제공: 고용노동부) 2025년 1월 1일부터 <b>육아</b>휴직급여가 월 최대 250만원으로 인상된다. 연간 지원 금액은 현행 1800만원에서 2310만원으로 오른다. 고용노동부는 17일 국무회의에서...', 'datetime': '2024-12-17T10:28:35.000+09:00', 'title': '내년부터 <b>육아</b>휴직급여 월 250만원 인상... 부부 연간 합산 5,920만원', 'url': 'https://www.bntnews.co.kr/article/view/bnt202412170083'}, {'contents': '넷플릭스가 기존에 제공하던 &#39;관대한&#39; <b>육아</b>정책이 점점 사라지고 있다. 12일(현지시간) 월스트리트저널(WJS)에 따르면 넷플릭스가 기존에 제공하던 휴가 혜택은 찾아보기 어려워졌다. 넷플릭스는 2015년 자녀를...', 'datetime': '2024-12-13T17:20:00.000+09:00', 'title': '사라진 넷플릭스의 &#39;1년 유급 <b>육아</b>휴직&#39;... “6개월 이상 쓰기 어려워” - 전자신문', 'url': 'https://www.etnews.com/20241213000225'}, {'contents': '2024 김천시 행복<b>육아</b> 사진 공모 당선작 전시 김천시는 3일부터 16일까지 김천시청 민원휴게실에서 행복<b>육아</b> 사진 공모 당선작을 전시한다. <b>육아</b>의 기쁨과 행복의 중요성을 알려 가족 친화적인 분위기를 조성...', 'datetime': '2024-12-04T21:19:57.000+09:00', 'title': '김천시 행복<b>육아</b> 사진 공모 당선작 전시', 'url': 'https://www.hgnet.co.kr/society/61497/'}, {'contents': '(비씨엔뉴스24) 지난 10년 <b>육아</b>휴직급여 수급자수는 81.1% 늘어났지만, 5인 미만 사업장에선 증가율이 전체 사업장 평균의 3분의 1에도 못 미치는 29.3%에 불과한 것으로 나타났다. <b>육아</b>휴직급여 수급에서도...', 'datetime': '2024-10-03T22:10:02.000+09:00', 'title': '김주영의원, ‘<b>육아</b>휴직’도 사업장 규모 따라 양극화...男 <b>육아</b>휴직, 10년새 &#39;15배&#39; 급증했지만 5...', 'url': 'http://www.bcnnews24.kr/news/article.html?no=250938'}, {'contents': '▲ 양평군<b>육아</b>종합지원센터는 어린이집연합회 임원진과 간담회 개최하고 애로사항을 청취했다. 【백운신문/양평방송】김명희 기자 = 경기 양평군<b>육아</b>종합지원센터(센터장 조연경)는 지난 12월 2일과 17일, 관내...', 'datetime': '2024-12-22T16:26:38.000+09:00', 'title': '양평군<b>육아</b>종합지원센터,22개 어린이집聯 임원진 애로청취 !', 'url': 'http://www.ypnews.kr/news/articleView.html?idxno=53110'}, {'contents': '또 초등학교 3학년에서 6학년 사이의 자녀를 키우는 공무원은 최대 36개월 범위 안에서 매일 1시간씩 <b>육아</b>시간을 사용하도록 하고, 직원들의 임신과 난임 시술을 위한 특별 휴가도 신설하기로 했습니다. 이장우...', 'datetime': '2024-12-11T00:00:00.000+09:00', 'title': '제목 주 4회만 출근! 대전시 출산‧<b>육아</b> 친화적 직장문화 만든다.', 'url': 'https://www.daejeon.go.kr/drh/board/boardNormalView.do?boardId=normal_1054&menuSeq=7629&pageIndex=1&ntatcSeq=1471656698'}, {'contents': '정부세종청사 인사혁신처. /뉴스1 ⓒ News1 장수영 기자 (서울=뉴스1) 정지형 기자 = 앞으로 공무원 <b>육아</b>휴직 전체 기간이 승진경력으로 인정된다. 인사혁신처는 31일 이 같은 내용을 담은 공무원임용령 개정안이...', 'datetime': '2024-12-31T17:31:02.000+09:00', 'title': '공무원 <b>육아</b>휴직 기간 전체 &#39;승진경력&#39; 인정…소급 적용', 'url': 'https://www.moneys.co.kr/article/2024123117318079460'}, {'contents': 'viewer 부산시청 전경. 사진제공=부산시 부산시는 소상공인을 대상으로 출산과 <b>육아</b> 부담을 덜어주는 ‘소상공인 <b>육아</b>응원패키지 지원사업’을 시행한다고 16일 밝혔다. 이 사업은 지난 9월 11일 부산시와 KB금융...', 'datetime': '2024-12-16T09:37:55.000+09:00', 'title': '출산·<b>육아</b> 대체인건비 등 부산 소상공인 출산·양육 지원', 'url': 'https://www.sedaily.com/NewsView/2DI56QOO6T/GK0208'}, {'contents': '(케이엠뉴스) 시흥시는 임산 부부를 대상으로 11월에도 ‘시흥형 출산ㆍ<b>육아</b>학교(4기)’를 정왕보건지소에서 운영하고 있다. 11월 강의는 지난 4일부터 시작돼 11일, 18일 총 3회에 걸쳐 진행되며, 회차별 20...', 'datetime': '2024-11-13T08:10:06.000+09:00', 'title': '건강한 출산과 <b>육아</b> 준비 도울 &#39;시흥형 출산·<b>육아</b>학교&#39; 4기 운영', 'url': 'http://www.kmnews.co.kr/news/article.html?no=302390'}], 'meta': {'is_end': False, 'pageable_count': 535, 'total_count': 1870807}}

,Title,Link,Description
0,<b>육아</b> - 나무위키,https://namu.wiki/w/%EC%9C%A1%EC%95%84,모유 수유하는 것이 생각보다 쉽지 않다. 울면서 분유 수유로 돌아서는 사람이 대부분...
1,워킹 맘 <b>육아</b> 대디 - 나무위키,https://namu.wiki/w/%EC%9B%8C%ED%82%B9%20%EB%A...,출산만 강요할 뿐 키우는 방법에 대해서는 함께 고민하지 않는 세상에서 부모라면 누구...
2,단독 2년간 줄줄 샌 ‘<b>육아</b>휴직 급여’ 54억 - 콘텐츠뷰,https://v.daum.net/v/KbZsnUVZng,https://n.news.naver.com/article/081/000351133...
3,&#34;내 삶은 어디에...&#34; <b>육아</b>에 지친 강아지 - 콘텐츠뷰,https://v.daum.net/v/VtboPtBWbv,"밤새 <b>육아</b>에 지친 강아지가 있습니다. 졸린 눈,, 떡진 머리, 피곤한 ..."
4,[🏛정책] ‘남성 <b>육아</b>휴직’ 1년새 20% 급증…작년 10명 중 3명 ...,https://www.etoland.co.kr/bbs/board.php?bo_tab...,https://www.donga.com/news/Economy/article/all...
5,[📰뉴스] <b>육아</b>휴직 양극화…10명 미만 중소기업 절반만 “모두 사용 가...,https://www.etoland.co.kr/bbs/board.php?bo_tab...,미디어오늘 https://www.mediatoday.co.kr https://www...
6,25년 <b>육아</b>휴직 관련 잘 아시는분 답변 부탁드려요,https://www.ppomppu.co.kr/zboard/view.php?id=b...,1. 25년 <b>육아</b>휴직 관련 유급 기간이나 금액이 변경됬는데 언제부터 시...
7,영유아 보육 &gt; 영유아 보육 지원 &gt; <b>육아</b> 근로 지원 &gt...,https://easylaw.go.kr/CSP/CnpClsMain.laf?csmSe...,사례로 알아볼 수 있도록 구성하였습니다. 아래에 항목을 선택하시면 내용을 보실 수 ...
8,<b>육아</b>휴직급여 이번년도에 완전 바뀌었어요! | 네이트 판,https://pann.nate.com/talk/373791699,2025년부터 대한민국의 <b>육아</b>휴직 급여 제도가 대폭 개선되었습니다. 가...
9,<b>육아</b>가 행복한 천안 ‘공동<b>육아</b>나눔터 10호점’ 개소,https://www.cheonan.go.kr/cop/bbs/BBSMSTR_0000...,- 백석LH아파트에 자녀돌봄 품앗이가 가능한 ‘공동<b>육아</b>나눔터’ 본격 운...


In [116]:
df_daum = daum_api("kpop", 5)
df_daum

,Title,Link,Description
0,K-POP - 나무위키,https://namu.wiki/w/K-POP,통용되고 있다. K-POP의 경우 특정 국가에서 시작되어 글로벌로 확산하기 까지의 ...
1,애플뮤직 에스파 2024 최고의 kpop 선정 - 콘텐츠뷰,https://v.daum.net/v/6kXu9lo9dx,캡처_2024_11_19_19_48_32_352.png 애플뮤직 에스파 2024 최...
2,"12살 캄보디아 공주, Kpop 아이돌이 되기 위해 한국행 소문 - 콘텐츠뷰",https://v.daum.net/v/4tCa5Vf5en,"20241015_063228.jpg 12살 캄보디아 공주, Kpop 아이돌이 되기 ..."
3,TEMPEST(아이돌)/KPOP MAKER - 나무위키,https://namu.wiki/w/TEMPEST(%EC%95%84%EC%9D%B4...,TEMPEST의 KPOP MAKER 콘텐츠를 서술한 문서이며 TEMPEST와 팬클럽...
4,"한투운용 ACE KPOP포커스 ETF, 최근 3개월 수익률 국내 주식형 중 1위",https://www.mydaily.co.kr/page/view/2024122308...,컴백 등 우호적 여건” /한국투자신탁운용 [마이데일리 = 신용승 기자] 한국투자신탁...


# Scrape

In [107]:
import pandas as pd
from datetime import datetime


def scrape(query:str, k:int, types:list[str], api:str) -> pd.DataFrame:
    today = datetime.today().strftime("%Y%m%d")

    df_google = google_api(query, k, types)
    df_google["search_engine"] = "Google"
    df_naver = naver_api(query, k, api)
    df_naver["search_engine"] = "Naver"
    df_daum = daum_api(query, k, api)
    df_daum["search_engine"] = "Daum"
    df_scrape= pd.concat([df_google, df_naver, df_daum])

    df_scrape['query'] = query
    df_scrape['fetched_date'] = today
    df_scrape.reset_index(inplace=True, drop=True)

    return df_scrape


def save2csv(df:pd.DataFrame) -> None:
    query = df["query"][0]
    query = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", query)
    date = df["fetched_date"][0]
    df.to_csv(f'{query}_{date}.csv', index=False, encoding="utf-8-sig")


In [109]:
query = "kpop"
k = 5
file_type_li = []
api = ""
df_scrape = scrape(query, k, file_type_li, api)
save2csv(df_scrape)
df_scrape

,Title,Link,Description,search_engine,query,fetched_date
0,K-pop - Wikipedia,https://en.wikipedia.org/wiki/K-pop,It includes styles and genres from around the ...,Google,kpop,20250116
1,Reddit K-Pop | Share and discover Korean music,https://www.reddit.com/r/kpop/,r/kpop: K-Pop (Korean popular music) is a musi...,Google,kpop,20250116
2,KPOP Junkee (@kpopjunkeecreative) • Instagram ...,https://www.instagram.com/kpopjunkeecreative/?...,"22K Followers, 657 Following, 14 Posts - KPOP ...",Google,kpop,20250116
3,Kpop Idol Unions: why is there no unionisation...,https://www.reddit.com/r/kpophelp/comments/1ea...,"Jul 23, 2024 ... A union is formed when worker...",Google,kpop,20250116
4,History - Ars Nova,https://arsnovanyc.com/about-us/history/,"... KPOP, created by Jason Kim, Max Vernon, He...",Google,kpop,20250116
5,K-POP - 나무위키,https://namu.wiki/w/K-POP,대한민국의 대중가요. 즉 Korean Popular Music에서 대중음악을 뜻하는...,Naver,kpop,20250116
6,K-pop - 위키백과 한국어,https://ko.wikipedia.org/wiki/K-pop,"K-pop ( /keɪ pɔp/ , 영어: Korean Popular Music ,...",Naver,kpop,20250116
7,K-POP RADAR 케이팝레이더,https://www.kpop-radar.com/,"오늘의 케이팝 현황, 팬덤 데이터 관측기. 아티스트별 유튜브, 트위터 등 온라인 팬...",Naver,kpop,20250116
8,케이팝(K-pop) - 한국민족문화대백과사전,https://encykorea.aks.ac.kr/Article/E0073667,• 본 항목의 내용은 해당 분야 전문가의 추천을 통해 선정된 집필자의 학술적 견해로...,Naver,kpop,20250116
9,K-pop - 한국방송예술진흥원,http://kbatv.org/dept/dept.asp?dcode=121,K-pop 전공소개 K-POP 전공은 현재 전세계적으로 각광 받고 있는 아이돌 음악...,Naver,kpop,20250116
